# Generate df_stats

In this notebook, I will collect the general statistics reported in the manuscript, from various sources.

Important note: for every % of reads displayed here, the percentage is calculated using the input number of reads for that given process. For example, imagine we start with 1000 reads. Then, 100 reads are filtered out due to some process, leaving 900. The survival rate is 90%. Then, these 900 reads are piped to the next process, which filters out another 200 reads. The survival rate is now 700/900 = 77.77%, and NOT 700/1000 = 70%!  
  
The reason for this choice is simple: It will be easier to isolate and compare a specific percentage between all methods without other filtering steps obfuscating the differences. If one is interested in the total number of reads surviving several steps, it's simply an issue of chaining each process's filtering rates together.

In [1]:
import pandas as pd
import polars as pl
import glob
import numpy as np
import os
import pickle

%load_ext lab_black

In [2]:
verbose = False

In [3]:
downsample_rate = "fixedcells"

I start by initializing a large data frame that will have all the stats.

In [4]:
samples = [
    x.split("/")[-1].split("__")[0]
    for x in sorted(glob.glob("../1_data_repository/libds_fastq/*R1*"))
]

In [5]:
if verbose == True:
    print(samples)

In [6]:
# df = pd.read_csv(samples)  # read the metadata file to know exactly which files were ran in the pipeline
# df_sample_names = df["sample_name"]
# if os.path.exists("fixedcells_general_statistics.tsv"):
#     df_stats = pd.read_csv("fixedcells_general_statistics.tsv", sep="\t", index_col=0)
# else:
df_stats = pd.DataFrame(index=samples)

Initialize some metadata

In [7]:
centre_dict = {
    "BIO": "BioRad",
    "BRO": "Broad",
    "CNA": "CNAG",
    "HAR": "Harvard",
    "MDC": "Max Delbrück Center",
    "SAN": "Wellcome Sanger Institute",
    "STA": "Stanford",
    "TXG": "10x Genomics",
    "UCS": "UCSF",
    "VIB": "VIB",
    "EPF": "EPFL",
    "OHS": "Oregon Health & Science University",
}

sample_id_ultrashort_alias_dict = {
    "BIO_ddseq_1": "ddS Bi1",
    "BIO_ddseq_2": "ddS Bi2",
    "BIO_ddseq_3": "ddS Bi3",
    "BIO_ddseq_4": "ddS Bi4",
    "BRO_mtscatac_1": "mt* Br1",
    "BRO_mtscatac_2": "mt* Br2",
    "CNA_10xmultiome_1": "MO C1",
    "CNA_10xmultiome_2": "MO C2",
    "CNA_10xv11_1": "v1.1 C1",
    "CNA_10xv11_2": "v1.1 C2",
    "CNA_10xv11_3": "v1.1 C3",
    "CNA_10xv11_4": "v1.1c C1",
    "CNA_10xv11_5": "v1.1c C2",
    "CNA_10xv2_1": "v2 C1",
    "CNA_10xv2_2": "v2 C2",
    "CNA_hydrop_1": "Hy C1",
    "CNA_hydrop_2": "Hy C2",
    "CNA_hydrop_3": "Hy C3",
    "CNA_mtscatac_1": "mt C1",
    "CNA_mtscatac_2": "mt C2",
    "EPF_hydrop_1": "Hy E1",
    "EPF_hydrop_2": "Hy E2",
    "EPF_hydrop_3": "Hy E3",
    "EPF_hydrop_4": "Hy E4",
    "HAR_ddseq_1": "ddS H1",
    "HAR_ddseq_2": "ddS H2",
    "MDC_mtscatac_1": "mt M1",
    "MDC_mtscatac_2": "mt M2",
    "OHS_s3atac_1": "s3 O1",
    "OHS_s3atac_2": "s3 O2",
    "SAN_10xmultiome_1": "MO Sa1",
    "SAN_10xmultiome_2": "MO Sa2",
    "STA_10xv11_1": "v1.1 St1",
    "STA_10xv11_2": "v1.1 St2",
    "TXG_10xv11_1": "v1.1 T1",
    "TXG_10xv2_1": "v2 T1",
    "TXG_10xv2_2": "v2 T2",
    "UCS_ddseq_1": "ddS U1",
    "UCS_ddseq_2": "ddS U2",
    "VIB_10xmultiome_1": "MO V1",
    "VIB_10xmultiome_2": "MO V2",
    "VIB_10xv1_1": "v1 V1",
    "VIB_10xv1_2": "v1 V2",
    "VIB_10xv2_1": "v2 V1",
    "VIB_10xv2_2": "v2 V2",
    "VIB_hydrop_1": "Hy V1",
    "VIB_hydrop_2": "Hy V2",
    "VIB_hydrop_11": "Hy V1",
    "VIB_hydrop_12": "Hy V1",
    "VIB_hydrop_21": "Hy V2",
    "VIB_hydrop_22": "Hy V2",
}

In [8]:
tech_dict = {
    "10xmultiome": "10x Multiome",
    "10xv1": "10x v1",
    "10xv11": "10x v1.1",
    "10xv2": "10x v2",
    "ddseq": "ddSEQ SureCell",
    "hydrop": "HyDrop",
    "mtscatac": "mtscATAC-seq",
    "s3atac": "s3-ATAC",
}

tech_alias_dict = {
    "10xmultiome": "10x Multiome",
    "10xv1": "10x v1",
    "10xv11": "10x v1.1",
    "10xv2": "10x v2",
    "ddseq": "Bio-Rad ddSEQ SureCell",
    "hydrop": "HyDrop",
    "mtscatac": "mtscATAC-seq",
    "s3atac": "s3-ATAC",
}

In [9]:
df_stats["short_identifier"] = [
    sample_id_ultrashort_alias_dict[x] for x in df_stats.index
]

In [10]:
df_stats["centre"] = [
    centre_dict[x] for x in np.array([x.split("_") for x in list(df_stats.index)])[:, 0]
]

In [11]:
df_stats["technology"] = [
    tech_alias_dict[x]
    for x in np.array([x.split("_") for x in list(df_stats.index)])[:, 1]
]

## Determine sequencer
I use this script: https://github.com/10XGenomics/supernova/blob/master/tenkit/lib/python/tenkit/illumina_instrument.py  
Very cool! Especially the in-built dict is nice:

```
# dictionary of instrument id regex: [platform(s)]
InstrumentIDs = {"HWI-M[0-9]{4}$" : ["MiSeq"],
        "HWUSI" : ["Genome Analyzer IIx"],
        "M[0-9]{5}$" : ["MiSeq"],
        "HWI-C[0-9]{5}$" : ["HiSeq 1500"],
        "C[0-9]{5}$" : ["HiSeq 1500"],
        "HWI-D[0-9]{5}$" : ["HiSeq 2500"],
        "D[0-9]{5}$" : ["HiSeq 2500"],
        "J[0-9]{5}$" : ["HiSeq 3000"],
        "K[0-9]{5}$" : ["HiSeq 3000","HiSeq 4000"],
        "E[0-9]{5}$" : ["HiSeq X"],
        "NB[0-9]{6}$": ["NextSeq"],
        "NS[0-9]{6}$" : ["NextSeq"],
        "MN[0-9]{5}$" : ["MiniSeq"]}

# dictionary of flow cell id regex: ([platform(s)], flow cell version and yeild)
FCIDs = {"C[A-Z,0-9]{4}ANXX$" : (["HiSeq 1500", "HiSeq 2000", "HiSeq 2500"], "High Output (8-lane) v4 flow cell"),
         "C[A-Z,0-9]{4}ACXX$" : (["HiSeq 1000", "HiSeq 1500", "HiSeq 2000", "HiSeq 2500"], "High Output (8-lane) v3 flow cell"),
         "H[A-Z,0-9]{4}ADXX$" : (["HiSeq 1500", "HiSeq 2500"], "Rapid Run (2-lane) v1 flow cell"),
         "H[A-Z,0-9]{4}BCXX$" : (["HiSeq 1500", "HiSeq 2500"], "Rapid Run (2-lane) v2 flow cell"),
         "H[A-Z,0-9]{4}BCXY$" : (["HiSeq 1500", "HiSeq 2500"], "Rapid Run (2-lane) v2 flow cell"),
         "H[A-Z,0-9]{4}BBXX$" : (["HiSeq 4000"], "(8-lane) v1 flow cell"),
         "H[A-Z,0-9]{4}BBXY$" : (["HiSeq 4000"], "(8-lane) v1 flow cell"),
         "H[A-Z,0-9]{4}CCXX$" : (["HiSeq X"], "(8-lane) flow cell"),
         "H[A-Z,0-9]{4}CCXY$" : (["HiSeq X"], "(8-lane) flow cell"),
         "H[A-Z,0-9]{4}ALXX$" : (["HiSeq X"], "(8-lane) flow cell"),
         "H[A-Z,0-9]{4}BGXX$" : (["NextSeq"], "High output flow cell"),
         "H[A-Z,0-9]{4}BGXY$" : (["NextSeq"], "High output flow cell"),
         "H[A-Z,0-9]{4}BGX2$" : (["NextSeq"], "High output flow cell"),
         "H[A-Z,0-9]{4}AFXX$" : (["NextSeq"], "Mid output flow cell"),
         "A[A-Z,0-9]{4}$" : (["MiSeq"], "MiSeq flow cell"),
         "B[A-Z,0-9]{4}$" : (["MiSeq"], "MiSeq flow cell"),
         "D[A-Z,0-9]{4}$" : (["MiSeq"], "MiSeq nano flow cell"),
         "G[A-Z,0-9]{4}$" : (["MiSeq"], "MiSeq micro flow cell"),
         "H[A-Z,0-9]{4}DMXX$" : (["NovaSeq"], "S2 flow cell")}

```

In [12]:
import gzip
import re

# dictionary of instrument id regex: [platform(s)]
InstrumentIDs = {
    "HWI-M[0-9]{4}$": ["MiSeq"],
    "HWUSI": ["Genome Analyzer IIx"],
    "M[0-9]{5}$": ["MiSeq"],
    "HWI-C[0-9]{5}$": ["HiSeq 1500"],
    "C[0-9]{5}$": ["HiSeq 1500"],
    "HWI-D[0-9]{5}$": ["HiSeq 2500"],
    "D[0-9]{5}$": ["HiSeq 2500"],
    "J[0-9]{5}$": ["HiSeq 3000"],
    "K[0-9]{5}$": ["HiSeq 3000", "HiSeq 4000"],
    "E[0-9]{5}$": ["HiSeq X"],
    "NB[0-9]{6}$": ["NextSeq 500/550"],
    "NS[0-9]{6}$": ["NextSeq 500/550"],
    "MN[0-9]{5}$": ["MiniSeq"],
    "N[0-9]{5}$": ["NextSeq 500/550"],  # added since original was outdated
    "A[0-9]{5}$": ["NovaSeq 6000"],  # added since original was outdated
    "V[0-9]{5}$": ["NextSeq 2000"],  # added since original was outdated
    "VH[0-9]{5}$": ["NextSeq 2000"],  # added since original was outdated
}

# dictionary of flow cell id regex: ([platform(s)], flow cell version and yeild)
FCIDs = {
    "C[A-Z,0-9]{4}ANXX$": (
        ["HiSeq 1500", "HiSeq 2000", "HiSeq 2500"],
        "High Output (8-lane) v4 flow cell",
    ),
    "C[A-Z,0-9]{4}ACXX$": (
        ["HiSeq 1000", "HiSeq 1500", "HiSeq 2000", "HiSeq 2500"],
        "High Output (8-lane) v3 flow cell",
    ),
    "H[A-Z,0-9]{4}ADXX$": (
        ["HiSeq 1500", "HiSeq 2500"],
        "Rapid Run (2-lane) v1 flow cell",
    ),
    "H[A-Z,0-9]{4}BCXX$": (
        ["HiSeq 1500", "HiSeq 2500"],
        "Rapid Run (2-lane) v2 flow cell",
    ),
    "H[A-Z,0-9]{4}BCXY$": (
        ["HiSeq 1500", "HiSeq 2500"],
        "Rapid Run (2-lane) v2 flow cell",
    ),
    "H[A-Z,0-9]{4}BBXX$": (["HiSeq 4000"], "(8-lane) v1 flow cell"),
    "H[A-Z,0-9]{4}BBXY$": (["HiSeq 4000"], "(8-lane) v1 flow cell"),
    "H[A-Z,0-9]{4}CCXX$": (["HiSeq X"], "(8-lane) flow cell"),
    "H[A-Z,0-9]{4}CCXY$": (["HiSeq X"], "(8-lane) flow cell"),
    "H[A-Z,0-9]{4}ALXX$": (["HiSeq X"], "(8-lane) flow cell"),
    "H[A-Z,0-9]{4}BGXX$": (["NextSeq"], "High output flow cell"),
    "H[A-Z,0-9]{4}BGXY$": (["NextSeq"], "High output flow cell"),
    "H[A-Z,0-9]{4}BGX2$": (["NextSeq"], "High output flow cell"),
    "H[A-Z,0-9]{4}AFXX$": (["NextSeq"], "Mid output flow cell"),
    "A[A-Z,0-9]{4}$": (["MiSeq"], "MiSeq flow cell"),
    "B[A-Z,0-9]{4}$": (["MiSeq"], "MiSeq flow cell"),
    "D[A-Z,0-9]{4}$": (["MiSeq"], "MiSeq nano flow cell"),
    "G[A-Z,0-9]{4}$": (["MiSeq"], "MiSeq micro flow cell"),
    "H[A-Z,0-9]{4}DMXX$": (["NovaSeq"], "S2 flow cell"),
}


SUPERNOVA_PLATFORM_BLACKLIST = ["HiSeq 3000", "HiSeq 4000", "HiSeq 3000/4000"]

_upgrade_set1 = set(["HiSeq 2000", "HiSeq 2500"])
_upgrade_set2 = set(["HiSeq 1500", "HiSeq 2500"])
_upgrade_set3 = set(["HiSeq 3000", "HiSeq 4000"])
_upgrade_set4 = set(["HiSeq 1000", "HiSeq 1500"])
_upgrade_set5 = set(["HiSeq 1000", "HiSeq 2000"])

fail_msg = "Cannot determine sequencing platform"
success_msg_template = "(likelihood: {})"
null_template = "{}"

# do intersection of lists
def intersect(a, b):
    return list(set(a) & set(b))


def union(a, b):
    return list(set(a) | set(b))


# extract ids from reads
def parse_readhead(head):
    fields = head.strip("\n").split(":")

    # if ill-formatted/modified non-standard header, return cry-face
    if len(fields) < 3:
        return -1, -1
    iid = fields[0][1:]
    fcid = fields[2]
    return iid, fcid


# infer sequencer from ids from single fastq
def infer_sequencer(iid, fcid):
    seq_by_iid = []
    for key in InstrumentIDs:
        if re.search(key, iid):
            seq_by_iid += InstrumentIDs[key]

    seq_by_fcid = []
    for key in FCIDs:
        if re.search(key, fcid):
            seq_by_fcid += FCIDs[key][0]

    sequencers = []

    # if both empty
    if not seq_by_iid and not seq_by_fcid:
        return sequencers, "fail"

    # if one non-empty
    if not seq_by_iid:
        return seq_by_fcid, "likely"
    if not seq_by_fcid:
        return seq_by_iid, "likely"

    # if neither empty
    sequencers = intersect(seq_by_iid, seq_by_fcid)
    if sequencers:
        return sequencers, "high"
    # this should not happen, but if both ids indicate different sequencers..
    else:
        sequencers = union(seq_by_iid, seq_by_fcid)
        return sequencers, "uncertain"


# process the flag and detected sequencer(s) for single fastq
def infer_sequencer_with_message(iid, fcid):
    sequencers, flag = infer_sequencer(iid, fcid)
    if not sequencers:
        return [""], fail_msg

    if flag == "high":
        msg_template = null_template
    else:
        msg_template = success_msg_template

    if set(sequencers) <= _upgrade_set1:
        return ["HiSeq2000/2500"], msg_template.format(flag)
    if set(sequencers) <= _upgrade_set2:
        return ["HiSeq1500/2500"], msg_template.format(flag)
    if set(sequencers) <= _upgrade_set3:
        return ["HiSeq3000/4000"], msg_template.format(flag)
    return sequencers, msg_template.format(flag)


def test_sequencer_detection():
    Samples = [
        "@ST-E00314:132:HLCJTCCXX:6:2206:31213:47966 1:N:0",
        "@D00209:258:CACDKANXX:6:2216:1260:1978 1:N:0:CGCAGTT",
        "@D00209:258:CACDKANXX:6:2216:1586:1970 1:N:0:GAGCAAG",
        "@A00311:74:HMLK5DMXX:1:1101:2013:1000 3:N:0:ACTCAGAC",
    ]

    seqrs = set()
    for head in Samples:
        iid, fcid = parse_readhead(head)
        seqr, msg = infer_sequencer_with_message(iid, fcid)
        for sr in seqr:
            signal = (sr, msg)
        seqrs.add(signal)

    print(seqrs)


def sequencer_detection_message(fastq_files):
    seqrs = set()
    # accumulate (sequencer, status) set
    for fastq in fastq_files:
        with gzip.open(fastq) as f:
            head = str(f.readline())
            # line = str(f.readline()
            # if len(line) > 0:
            #     if line[0] == "@":
            #         head = line
            #     else:
            #         print("Incorrectly formatted first read in FASTQ file: %s" % fastq)
            #         print(line)

        iid, fcid = parse_readhead(head)
        seqr, msg = infer_sequencer_with_message(iid, fcid)
        for sr in seqr:
            signal = (sr, msg)
        seqrs.add(signal)

    # get a list of sequencing platforms
    platforms = set()
    for platform, _ in seqrs:
        platforms.add(platform)
    sequencers = list(platforms)

    # if no sequencer detected at all
    message = ""
    fails = 0
    for platform, status in seqrs:
        if status == fail_msg:
            fails += 1
    if fails == len(seqrs):
        message = "could not detect the sequencing platform(s) used to generate the input FASTQ files"
        return message, sequencers

    # if partial or no detection failures
    if fails > 0:
        message = "could not detect the sequencing platform used to generate some of the input FASTQ files, "
    message += "detected the following sequencing platforms- "
    for platform, status in seqrs:
        if status != fail_msg:
            message += platform + " " + status + ", "
    message = message.strip(", ")
    return message, sequencers

In [13]:
sequencers_dict = {}
for file in sorted(list(glob.glob("../1_data_repository/libds_fastq/*R2*.fastq.gz"))):
    filename = file.split("/")[-1]
    samplename = filename.split(".")[0].split("__")[0]
    if (
        samplename.split("_")[1] == "s3atac"
    ):  # s3atac fastqs have altered readname due to unidex preprocessing, the original fastqs show that they were sequenced on NS2000.
        sequencers = ["NovaSeq 6000"]
    else:
        message, sequencers = sequencer_detection_message([file])
    print(f"{filename}: {sequencers}")

    sequencers_dict[samplename] = sequencers[0]

BIO_ddseq_1__R2.LIBDS.fastq.gz: ['NovaSeq']
BIO_ddseq_2__R2.LIBDS.fastq.gz: ['NovaSeq']
BIO_ddseq_3__R2.LIBDS.fastq.gz: ['NextSeq 500/550']
BIO_ddseq_4__R2.LIBDS.fastq.gz: ['NextSeq 500/550']
BRO_mtscatac_1__R2.LIBDS.fastq.gz: ['NextSeq 500/550']
BRO_mtscatac_2__R2.LIBDS.fastq.gz: ['NextSeq 500/550']
CNA_10xmultiome_1__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xmultiome_2__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv11_1__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv11_2__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv11_3__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv11_4__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv11_5__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv2_1__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_10xv2_2__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_hydrop_1__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_hydrop_2__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_hydrop_3__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_mtscatac_1__R2.LIBDS.fastq.gz: ['NovaSeq 6000']
CNA_mtscatac_2__R2.LIBDS.fastq.gz: ['N

In [14]:
df_stats["sequencing_instrument"] = df_stats.index.map(sequencers_dict)

## Determine read counts

In [15]:
read_count_df = pd.DataFrame()
for readfile in sorted(glob.glob("../1_data_repository/R*_lengths.LIBDS.sorted.txt")):
    read = readfile.split("/")[-1].split("_")[0]
    df = pd.read_csv(readfile, sep="\t", header=None)
    df.index = [x.split("/")[-1].split("__")[0] for x in df[0]]
    df = df[1]
    read_count_df[read] = df

In [16]:
if verbose == True:
    print(read_count_df)

In [17]:
read_dict = {}
for line in read_count_df.index:
    if read_count_df.isna().loc[line]["R3"]:
        if read_count_df.loc[line]["R1"] == read_count_df.loc[line]["R2"]:
            print(f"{line} OK")
            read_dict[line] = read_count_df.loc[line]["R1"]
        else:
            print(f"{line} has read discrepancy!")
            print(f'\t{read_count_df.loc[line]["R1"]}')
            print(f'\t{read_count_df.loc[line]["R2"]}')
    else:
        if (
            read_count_df.loc[line]["R1"]
            == read_count_df.loc[line]["R2"]
            == read_count_df.loc[line]["R3"]
        ):
            print(f"{line} OK")
            read_dict[line] = read_count_df.loc[line]["R1"]
        else:
            print(f"{line} has read discrepancy!")
            print(f'\t{read_count_df.loc[line]["R1"]}')
            print(f'\t{read_count_df.loc[line]["R2"]}')
            print(f'\t{read_count_df.loc[line]["R3"]}')

BIO_ddseq_1 OK
BIO_ddseq_2 OK
BIO_ddseq_3 OK
BIO_ddseq_4 OK
BRO_mtscatac_1 OK
BRO_mtscatac_2 OK
CNA_10xmultiome_1 OK
CNA_10xmultiome_2 OK
CNA_10xv11_1 OK
CNA_10xv11_2 OK
CNA_10xv11_3 OK
CNA_10xv11_4 OK
CNA_10xv11_5 OK
CNA_10xv2_1 OK
CNA_10xv2_2 OK
CNA_hydrop_1 OK
CNA_hydrop_2 OK
CNA_hydrop_3 OK
CNA_mtscatac_1 OK
CNA_mtscatac_2 OK
EPF_hydrop_1 OK
EPF_hydrop_2 OK
EPF_hydrop_3 OK
EPF_hydrop_4 OK
HAR_ddseq_1 OK
HAR_ddseq_2 OK
MDC_mtscatac_1 OK
MDC_mtscatac_2 OK
OHS_s3atac_1 OK
OHS_s3atac_2 OK
SAN_10xmultiome_1 OK
SAN_10xmultiome_2 OK
STA_10xv11_1 OK
STA_10xv11_2 OK
TXG_10xv11_1 OK
TXG_10xv2_1 OK
TXG_10xv2_2 OK
UCS_ddseq_1 OK
UCS_ddseq_2 OK
VIB_10xmultiome_1 OK
VIB_10xmultiome_2 OK
VIB_10xv1_1 OK
VIB_10xv1_2 OK
VIB_10xv2_1 OK
VIB_10xv2_2 OK
VIB_hydrop_11 OK
VIB_hydrop_12 OK
VIB_hydrop_21 OK
VIB_hydrop_22 OK


In [18]:
df_stats["reads"] = df_stats.index.map(read_dict)

In [19]:
if verbose == True:
    print(df_stats)

# Determine cell counts

In [20]:
tsv_list = sorted(
    glob.glob(f"../{downsample_rate}_2_cistopic/selected_barcodes/*.RAW.txt")
)
cell_count_dict = {}
for tsv in tsv_list:
    sample = tsv.split("/")[-1].split(".")[0].split("_metadata_bc_df")[0]
    # print(sample)
    df = pd.read_csv(tsv, sep="\t", index_col=0)
    cell_count_dict[sample] = len(df)

df_stats["cells"] = df_stats.index.map(cell_count_dict)
df_stats["cells"]

BIO_ddseq_1           6359
BIO_ddseq_2           5159
BIO_ddseq_3           2801
BIO_ddseq_4           2649
BRO_mtscatac_1        3575
BRO_mtscatac_2        3398
CNA_10xmultiome_1     3536
CNA_10xmultiome_2     3122
CNA_10xv11_1          2733
CNA_10xv11_2          2785
CNA_10xv11_3          4603
CNA_10xv11_4           803
CNA_10xv11_5          1111
CNA_10xv2_1           4179
CNA_10xv2_2           5994
CNA_hydrop_1          1770
CNA_hydrop_2          2177
CNA_hydrop_3          1417
CNA_mtscatac_1        2633
CNA_mtscatac_2        1169
EPF_hydrop_1          3251
EPF_hydrop_2          3104
EPF_hydrop_3          2863
EPF_hydrop_4          2856
HAR_ddseq_1           4661
HAR_ddseq_2           5079
MDC_mtscatac_1        7990
MDC_mtscatac_2        5628
OHS_s3atac_1          3120
OHS_s3atac_2          1787
SAN_10xmultiome_1     3492
SAN_10xmultiome_2     4269
STA_10xv11_1           934
STA_10xv11_2          1571
TXG_10xv11_1          9933
TXG_10xv2_1           9977
TXG_10xv2_2          10005
U

Now, calculate the RPC:

In [21]:
df_stats["RPC"] = df_stats["reads"] / df_stats["cells"]

## Determine sequence lengths

In [22]:
## Determine average insert length

### Barcode correction stats

In [23]:
directory = "../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/barcode/"
for sample in df_stats.index:
    file = directory + sample + "_____R1.corrected.bc_stats.log"
    if os.path.exists(file):
        # print(f"{sample}: {file}")
        df = pd.read_csv(file, sep="\t\t|\t", engine="python", index_col=0, header=None)
        # print(df)
        if "ddseq" in sample:
            nreads = df.loc["nbr_reads:", 1]
            nbarcodes_total = df.loc[
                "nbr_reads_with_bc1_bc2_bc3_correct_or_correctable", 1
            ]
            percentage_correct_barcodes = nbarcodes_total / nreads * 100
        else:
            nreads = df.loc["nbr_reads:", 1]
            nbarcodes_total = df.loc["total_bc_found", 1]
            percentage_correct_barcodes = nbarcodes_total / nreads * 100

        if verbose == True:
            print(f"nreads: {nreads}")
            print(f"nbarcodes_total: {nbarcodes_total}")
            print(f"percentage_correct_barcodes: {percentage_correct_barcodes}")
            print("-------------------------------------\n")

        df_stats.loc[sample, "nreads"] = int(nreads)
        df_stats.loc[sample, "%_correct_barcodes"] = round(
            percentage_correct_barcodes, 2
        )
    else:
        print(f"{file} does not exist!")

In [24]:
df_stats

,short_identifier,centre,technology,sequencing_instrument,reads,cells,RPC,nreads,%_correct_barcodes
BIO_ddseq_1,ddS Bi1,BioRad,Bio-Rad ddSEQ SureCell,NovaSeq,259455742.0,6359,40801.343293,259455742.0,96.54
BIO_ddseq_2,ddS Bi2,BioRad,Bio-Rad ddSEQ SureCell,NovaSeq,210510517.0,5159,40804.519674,210510517.0,96.76
BIO_ddseq_3,ddS Bi3,BioRad,Bio-Rad ddSEQ SureCell,NextSeq 500/550,114310431.0,2801,40810.578722,114310431.0,87.47
BIO_ddseq_4,ddS Bi4,BioRad,Bio-Rad ddSEQ SureCell,NextSeq 500/550,108112191.0,2649,40812.454134,108112191.0,88.10
BRO_mtscatac_1,mt* Br1,Broad,mtscATAC-seq,NextSeq 500/550,145886207.0,3575,40807.330629,145886207.0,94.66
BRO_mtscatac_2,mt* Br2,Broad,mtscATAC-seq,NextSeq 500/550,138669154.0,3398,40809.050618,138669154.0,94.79
CNA_10xmultiome_1,MO C1,CNAG,10x Multiome,NovaSeq 6000,144290077.0,3536,40806.017251,144290077.0,98.31
CNA_10xmultiome_2,MO C2,CNAG,10x Multiome,NovaSeq 6000,127397732.0,3122,40806.448430,127397732.0,98.40
CNA_10xv11_1,v1.1 C1,CNAG,10x v1.1,NovaSeq 6000,111532272.0,2733,40809.466520,111532272.0,98.19
CNA_10xv11_2,v1.1 C2,CNAG,10x v1.1,NovaSeq 6000,113657151.0,2785,40810.467145,113657151.0,98.17


## Mapping statistics

In [25]:
directory = (
    "../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/"
)

for sample in df_stats.index:
    file = directory + sample + "_____R1.mapping_stats.tsv"
    if os.path.exists(file):
        print(f"{sample}: {file}")
        df = pd.read_csv(file, sep="\t", engine="python", index_col=0, header=0)
        if verbose == True:
            print(df.astype(int))
            print("\n")

        percent_mapq30 = (
            df.loc["Reads mapped with MAPQ>30:"] / df.loc["raw total sequences:"] * 100
        )
        avg_insert = df.loc["insert size average:"]
        avg_map_quality = df.loc["average quality:"]
        r1_length = df.loc["maximum first fragment length:"]
        r2_length = df.loc["maximum last fragment length:"]

        if verbose == True:

            print(f"read 1 length: {int(r1_length)}")
            print(f"read 2 length: {int(r2_length)}")
            print(f"average map quality: {round(avg_map_quality, 2)}")
            print(f"percent mapq30: {round(percent_mapq30, 2)}")
            print(f"insert size average: {avg_insert}")
            print("-------------------------------------\n")

        df_stats.loc[sample, "r1_length"] = int(r1_length)
        df_stats.loc[sample, "r2_length"] = int(r2_length)
        df_stats.loc[sample, "avg_insert_size"] = int(avg_insert)
        df_stats.loc[sample, "%_mapq30"] = round(percent_mapq30.iloc[0], 2)
        df_stats.loc[sample, "avg_map_quality"] = round(avg_map_quality.iloc[0], 2)
    elif verbose == True:
        print(f"{file}")

BIO_ddseq_1: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BIO_ddseq_1_____R1.mapping_stats.tsv
BIO_ddseq_2: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BIO_ddseq_2_____R1.mapping_stats.tsv
BIO_ddseq_3: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BIO_ddseq_3_____R1.mapping_stats.tsv
BIO_ddseq_4: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BIO_ddseq_4_____R1.mapping_stats.tsv
BRO_mtscatac_1: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BRO_mtscatac_1_____R1.mapping_stats.tsv
BRO_mtscatac_2: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/BRO_mtscatac_2_____R1.mapping_stats.tsv
CNA_10xmultiome_1: ../libds_1_vsn_preprocessing/libds_preprocessing_out/data/reports/mapping_stats/CNA_10xmultiome_1_____R1.mapping_stats.tsv
CNA_10xmultiome_2: ../libds_1_vsn_preprocessing/libds_preprocessin

In [26]:
for hydrop_number in ["VIB_hydrop_1", "VIB_hydrop_2"]:
    df_stats.loc[f"{hydrop_number}"] = df_stats.loc[f"{hydrop_number}1"]

    # additive
    for var in ["reads", "nreads", "cells"]:
        df_stats.at[f"{hydrop_number}", var] = (
            df_stats.loc[f"{hydrop_number}1"][var]
            + df_stats.loc[f"{hydrop_number}2"][var]
        )

    # weighted average
    weight_1 = df_stats.loc[f"{hydrop_number}1"]["reads"] / (
        df_stats.loc[f"{hydrop_number}1"]["reads"]
        + df_stats.loc[f"{hydrop_number}2"]["reads"]
    )
    weight_2 = df_stats.loc[f"{hydrop_number}2"]["reads"] / (
        df_stats.loc[f"{hydrop_number}1"]["reads"]
        + df_stats.loc[f"{hydrop_number}2"]["reads"]
    )

    for var in ["avg_insert_size", "avg_map_quality", "%_correct_barcodes", "%_mapq30"]:
        df_stats.at[f"{hydrop_number}", var] = (
            df_stats.loc[f"{hydrop_number}1"][var] * weight_1
            + df_stats.loc[f"{hydrop_number}2"][var] * weight_2
        )

    # special
    var = "RPC"
    df_stats.at[f"{hydrop_number}", var] = (
        df_stats.loc[f"{hydrop_number}"]["reads"]
        / df_stats.loc[f"{hydrop_number}"]["cells"]
    )

In [27]:
df_stats.columns

Index(['short_identifier', 'centre', 'technology', 'sequencing_instrument',
       'reads', 'cells', 'RPC', 'nreads', '%_correct_barcodes', 'r1_length',
       'r2_length', 'avg_insert_size', '%_mapq30', 'avg_map_quality'],
      dtype='object')

In [28]:
df_stats

,short_identifier,centre,technology,sequencing_instrument,reads,cells,RPC,nreads,%_correct_barcodes,r1_length,r2_length,avg_insert_size,%_mapq30,avg_map_quality
BIO_ddseq_1,ddS Bi1,BioRad,Bio-Rad ddSEQ SureCell,NovaSeq,259455742.0,6359,40801.343293,259455742.0,96.540000,53.0,40.0,172.000000,90.500000,36.100000
BIO_ddseq_2,ddS Bi2,BioRad,Bio-Rad ddSEQ SureCell,NovaSeq,210510517.0,5159,40804.519674,210510517.0,96.760000,52.0,40.0,167.000000,92.140000,36.100000
BIO_ddseq_3,ddS Bi3,BioRad,Bio-Rad ddSEQ SureCell,NextSeq 500/550,114310431.0,2801,40810.578722,114310431.0,87.470000,53.0,40.0,129.000000,87.210000,31.500000
BIO_ddseq_4,ddS Bi4,BioRad,Bio-Rad ddSEQ SureCell,NextSeq 500/550,108112191.0,2649,40812.454134,108112191.0,88.100000,54.0,40.0,129.000000,87.250000,31.400000
BRO_mtscatac_1,mt* Br1,Broad,mtscATAC-seq,NextSeq 500/550,145886207.0,3575,40807.330629,145886207.0,94.660000,72.0,72.0,138.000000,87.550000,33.100000
BRO_mtscatac_2,mt* Br2,Broad,mtscATAC-seq,NextSeq 500/550,138669154.0,3398,40809.050618,138669154.0,94.790000,72.0,72.0,137.000000,87.660000,33.100000
CNA_10xmultiome_1,MO C1,CNAG,10x Multiome,NovaSeq 6000,144290077.0,3536,40806.017251,144290077.0,98.310000,50.0,49.0,160.000000,89.650000,36.100000
CNA_10xmultiome_2,MO C2,CNAG,10x Multiome,NovaSeq 6000,127397732.0,3122,40806.448430,127397732.0,98.400000,50.0,49.0,165.000000,89.360000,36.100000
CNA_10xv11_1,v1.1 C1,CNAG,10x v1.1,NovaSeq 6000,111532272.0,2733,40809.466520,111532272.0,98.190000,50.0,49.0,161.000000,90.030000,36.400000
CNA_10xv11_2,v1.1 C2,CNAG,10x v1.1,NovaSeq 6000,113657151.0,2785,40810.467145,113657151.0,98.170000,50.0,49.0,157.000000,90.100000,36.400000


# single_cell_stats

In [29]:
metadata_path_dict = {
    x.split("/")[-1].split(f"__")[0].split(".")[0]: x
    for x in sorted(
        glob.glob(
            f"../{downsample_rate}_3_cistopic_consensus/cistopic_qc_out_CONSENSUS/*metadata*pkl"
        )
    )
}
if verbose:
    print(metadata_path_dict)

In [30]:
selected_cells_path_dict = {
    x.split("/")[-1].split(f"__")[0].split(".")[0]: x
    for x in sorted(
        glob.glob(f"../{downsample_rate}_3_cistopic_consensus/selected_barcodes/*.pkl")
    )
}
if verbose:
    print(selected_cells_path_dict)

In [31]:
df_merged = pd.DataFrame()
for sample in metadata_path_dict.keys():
    with open(metadata_path_dict[sample], "rb") as f:
        df = pickle.load(f)

    with open(selected_cells_path_dict[sample], "rb") as f:
        selected_barcodes = pickle.load(f)

    if downsample_rate == "fixedcells":
        selected_barcodes = [x.replace("FULL", "FIXEDCELLS") for x in selected_barcodes]

    df = df.loc[selected_barcodes]
    df_median = df.median()
    df_median.index = ["Median_" + x.lower() for x in df_median.index]
    df_median["total_nr_frag_in_selected_barcodes"] = sum(df["Total_nr_frag"])
    df_median["total_nr_unique_frag_in_selected_barcodes"] = sum(df["Unique_nr_frag"])
    df_median["total_nr_unique_frag_in_selected_barcodes_in_regions"] = sum(
        df["Unique_nr_frag_in_regions"]
    )
    df_median["n_barcodes_merged"] = len(
        [x for x in [x.split("__")[0] for x in df.index] if "_" in x]
    )
    df_median["frac_barcodes_merged"] = len(
        [x for x in [x.split("__")[0] for x in df.index] if "_" in x]
    ) / len(df)
    df_merged = pd.concat([df_merged, df_median], axis=1)

/tmp/ipykernel_911774/911950526.py:13: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_median = df.median()
/tmp/ipykernel_911774/911950526.py:13: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_median = df.median()
/tmp/ipykernel_911774/911950526.py:13: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warnin

In [32]:
df_merged.columns = metadata_path_dict.keys()

df_merged = df_merged.T

In [33]:
df_merged = df_merged[
    [
        "Median_total_nr_frag",
        "Median_unique_nr_frag",
        "Median_dupl_rate",
        "Median_total_nr_frag_in_regions",
        "Median_frip",
        "Median_tss_enrichment",
        "total_nr_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes_in_regions",
        "n_barcodes_merged",
        "frac_barcodes_merged",
    ]
]

In [34]:
if not "total_nr_frag_in_selected_barcodes" in df_stats.columns:
    df_stats = pd.concat([df_stats, df_merged], axis=1)

In [35]:
df_stats = df_stats.loc[:, ~df_stats.columns.duplicated()].copy()

In [36]:
df_stats.columns

Index(['short_identifier', 'centre', 'technology', 'sequencing_instrument',
       'reads', 'cells', 'RPC', 'nreads', '%_correct_barcodes', 'r1_length',
       'r2_length', 'avg_insert_size', '%_mapq30', 'avg_map_quality',
       'Median_total_nr_frag', 'Median_unique_nr_frag', 'Median_dupl_rate',
       'Median_total_nr_frag_in_regions', 'Median_frip',
       'Median_tss_enrichment', 'total_nr_frag_in_selected_barcodes',
       'total_nr_unique_frag_in_selected_barcodes',
       'total_nr_unique_frag_in_selected_barcodes_in_regions',
       'n_barcodes_merged', 'frac_barcodes_merged'],
      dtype='object')

In [37]:
df_stats["efficiency"] = (
    df_stats["total_nr_unique_frag_in_selected_barcodes_in_regions"] / df_stats["reads"]
)

# Count total fragments

In [38]:
fragments_path_dict = {
    x.split("/")[-1].split(f"__")[0].split(".")[0]: x
    for x in sorted(
        glob.glob(f"../1_data_repository/{downsample_rate}_fragments/*.tsv.gz")
    )
}
if verbose:
    print(fragments_path_dict)

In [39]:
chroms_standard = (
    ["chr" + str(x + 1) for x in range(22)] + ["chrX"] + ["chrY"] + ["chrM"]
)
counts_in_standard_chrom_df = pl.DataFrame({"Chromosome": chroms_standard})
count_len_dict = {}
if not "total_fragments" in df_stats.columns:
    if os.path.exists("fixedcells_general_chroms.csv"):
        df_frags = pd.read_csv("fixedcells_general_chroms.csv", index_col=0)
        df_stats = pd.concat([df_stats, df_frags], axis=1)

    else:
        for sample in fragments_path_dict:
            print(sample)

            df = pl.read_csv(
                fragments_path_dict[sample], has_header=False, sep="\t", n_threads=4
            )

            df.columns = ["Chromosome", "Start", "End", "CB", "Count"]
            df_counts_per_chrom = (
                df
                # .filter(pl.col("CB").is_in(selected_CBs)
                .groupby(["Chromosome", "Start", "End", "CB"])
                .agg(
                    [
                        pl.col("Chromosome").first().alias("temp"),
                    ]
                )
                .groupby("Chromosome")
                .agg(pl.count().alias("count"))
                .sort(by="Chromosome")
            )

            total_counts_in_standard_chroms = df_counts_per_chrom.sum()
            total_counts_in_standard_chroms = df_counts_per_chrom.filter(
                pl.col("Chromosome").is_in(chroms_standard)
            ).sum()

            counts_in_standard_chroms = df_counts_per_chrom.filter(
                pl.col("Chromosome").is_in(chroms_standard)
            )
            counts_in_standard_chroms = counts_in_standard_chroms.rename(
                {"count": sample}
            )
            counts_in_standard_chrom_df = counts_in_standard_chrom_df.join(
                counts_in_standard_chroms, on="Chromosome"
            )
            count_len_dict[sample] = len(df)

        df = counts_in_standard_chrom_df.to_pandas().T
        df.columns = df.loc["Chromosome"]
        df = df.drop("Chromosome")

        for sample in df.index:
            df.at[sample, "total"] = count_len_dict[sample]

        df["nonstandard"] = df["total"] - df[df.columns[:-1]].sum(axis=1)

        df = df.div(df["total"], axis=0) * 100

        for sample in df.index:
            df.at[sample, "total_fragments"] = count_len_dict[sample]

        df.drop("total", inplace=True, axis="columns")

        df_stats = pd.concat([df_stats, df], axis=1)

else:
    print("already done")

# add dar strength and count

In [40]:
dar_path_dict = {
    x.split("/")[-1]: x
    for x in sorted(
        glob.glob(
            "../fixedcells_3_cistopic_consensus/downstream_analysis/DARs/*/*DARs.bed"
        )
    )
}
if verbose:
    print(dar_path_dict)

In [41]:
df = pd.DataFrame(index=df_stats.index)
if not "n_dars__B_cell" in df_stats.columns:
    for sample, dar_path in dar_path_dict.items():
        supersample = sample.split(".")[0]
        cell_type = sample.split("__")[1]
        if verbose:
            print(sample)
            print(cell_type)
        col_name = "n_dars__" + cell_type

        df_dars = pd.read_csv(dar_path, sep="\t", header=None)
        df.at[supersample, col_name] = len(df_dars)

        col_name = "top_2000_dars_median_logfc__" + cell_type
        df.at[supersample, col_name] = df_dars[4][0:2000].median()

        col_name = "top_2000_dars_median_fc__" + cell_type
        df.at[supersample, col_name] = 2 ** df_dars[4][0:2000].median()

# add peak strength and count

In [42]:
peak_path_dict = {
    x.split("/")[-2] + "__" + x.split("/")[-1]: x
    for x in sorted(
        glob.glob(
            "../fixedcells_3_cistopic_consensus/final_consensus_peaks/*__SCREEN_consensus_peaks/*narrowPeak"
        )
    )
}
if verbose:
    print(peak_path_dict)

In [43]:
cell_type

'Cytotoxic_T_cell'

In [44]:
for sample, dar_path in peak_path_dict.items():
    if verbose:
        print(sample)
    supersample = sample.split(".")[0]
    cell_type = sample.split("__")[-1].split(".")[0].split("_peaks")[0]
    if verbose:
        print(cell_type)
    col_name = "n_peaks__" + cell_type

    df_peaks = pd.read_csv(dar_path, sep="\t", header=None)
    df.at[supersample, col_name] = len(df_peaks)

    col_name = "top10k_peaks_strength__" + cell_type
    df.at[supersample, col_name] = df_peaks[6][0:10000].median()
    col_name = "top10k_peaks_pval" + cell_type
    df.at[supersample, col_name] = df_peaks[7][0:10000].median()
    col_name = "top10k_peaks_qval" + cell_type
    df.at[supersample, col_name] = df_peaks[8][0:10000].median()

In [45]:
df_peaks

,0,1,2,3,4,5,6,7,8,9
0,chr1,629195,629317,CytotoxicTcell_peak_1,11519,.,337.11000,1157.29000,1151.99000,76
1,chr1,629908,630058,CytotoxicTcell_peak_2,3223,.,112.66600,327.25200,322.39500,37
2,chr1,633254,633421,CytotoxicTcell_peak_3,25,.,3.54853,5.00599,2.52812,44
3,chr1,633941,634096,CytotoxicTcell_peak_4,54565,.,1331.58000,5461.93000,5456.59000,84
4,chr1,778487,779114,CytotoxicTcell_peak_5,1808,.,69.19630,184.95100,180.84000,244
...,...,...,...,...,...,...,...,...,...,...
52310,chrY,19077416,19077584,CytotoxicTcell_peak_43921,73,.,6.20992,10.01870,7.30265,92
52311,chrY,19567084,19567498,CytotoxicTcell_peak_43922a,108,.,7.98419,13.66820,10.84690,95
52312,chrY,19567084,19567498,CytotoxicTcell_peak_43922b,205,.,12.41980,23.52780,20.51960,254
52313,chrY,19567717,19567946,CytotoxicTcell_peak_43923,288,.,15.96840,31.97620,28.85860,116


In [46]:
if not "n_peaks__B_cell" in df_stats.columns:
    df_stats = pd.concat([df_stats, df], axis=1)

# add recovered % DARs and peaks recovered

In [47]:
import pandas as pd

In [48]:
df = pd.read_csv(
    "../fixedcells_8_individual_tech_cistopic_objects/peak_dar_recovery_individual_samples.tsv",
    sep="\t",
    index_col=0,
)

In [49]:
df

,B_cells_bot20peaks_recovery,Naive_T_cells_bot20peaks_recovery,Cytotoxic_T_cells_bot20peaks_recovery,NK_cells_bot20peaks_recovery,CD14+_monocytes_bot20peaks_recovery,CD16+_monocytes_bot20peaks_recovery,Dendritic_cells_bot20peaks_recovery,mean_bot20peaks_recovery,B_cells_top20peaks_recovery,Naive_T_cells_top20peaks_recovery,...,Natural_killer_cell,mean_bot20dars_recovery,B_cell_top20dars_recovery,CD4+_T_cell_top20dars_recovery,Cytotoxic_T_cell_top20dars_recovery,Natural_killer_cell_top20dars_recovery,CD14+_monocyte_top20dars_recovery,CD16+_monocyte_top20dars_recovery,Dendritic_cell_top20dars_recovery,mean_top20dars_recovery
sample,,,,,,,,,,,,,,,,,,,,,
BIO_ddseq_1,0.048732,0.054490,0.005072,0.032241,0.073321,NaN,0.033081,0.041156,0.875821,0.922818,...,0.224573,0.280102,0.248372,0.329651,0.282447,0.360410,0.349878,NaN,0.334588,0.317558
BIO_ddseq_2,0.058525,0.106334,0.005103,0.040168,0.059927,NaN,0.021725,0.048630,0.891785,0.938597,...,0.308532,0.291218,0.231163,0.305814,0.239362,0.322184,0.399919,NaN,0.304057,0.300416
BIO_ddseq_3,0.029213,0.024504,0.007033,0.010109,0.087114,0.048462,0.007077,0.030501,0.855937,0.911619,...,0.143345,0.422531,0.334419,0.397093,0.172872,0.108532,0.764849,0.432505,0.554161,0.394919
BIO_ddseq_4,0.033727,0.026697,0.008433,0.014046,0.040793,0.041192,0.011027,0.025131,0.871296,0.917942,...,0.127645,0.350729,0.377674,0.424419,0.137766,0.077816,0.794752,0.443189,0.221455,0.353867
BRO_mtscatac_1,0.046508,0.045302,0.004139,0.076452,0.119403,0.138792,0.080151,0.072964,0.979192,0.994179,...,0.688737,0.640961,0.552093,0.917442,0.864894,0.845051,0.973759,0.961167,0.769971,0.840625
BRO_mtscatac_2,0.020548,0.042050,0.005134,0.058523,0.098036,0.106447,0.029954,0.051527,0.957970,0.994770,...,0.643686,0.648653,0.481860,0.919186,0.877660,0.857338,0.970098,0.981303,0.808239,0.842240
CNA_10xmultiome_1,0.060284,0.118472,0.006784,0.056501,0.072338,NaN,0.037196,0.058596,0.992703,0.998582,...,0.517406,0.569420,0.820465,0.805233,0.851596,0.869625,0.878153,NaN,0.813885,0.839826
CNA_10xmultiome_2,0.030474,0.096578,0.011016,0.043254,0.069308,NaN,0.018378,0.044835,0.946912,0.997902,...,0.337884,0.438058,0.637674,0.768605,0.633511,0.645734,0.910903,NaN,0.846508,0.740489
CNA_10xv11_1,0.017494,0.065154,0.008682,0.073207,0.020410,0.035107,NaN,0.036676,0.956121,0.996247,...,0.604778,0.627695,0.907442,0.913372,0.893617,0.879863,0.888731,0.935895,NaN,0.903153


In [50]:
list(df_stats.columns)

['short_identifier',
 'centre',
 'technology',
 'sequencing_instrument',
 'reads',
 'cells',
 'RPC',
 'nreads',
 '%_correct_barcodes',
 'r1_length',
 'r2_length',
 'avg_insert_size',
 '%_mapq30',
 'avg_map_quality',
 'Median_total_nr_frag',
 'Median_unique_nr_frag',
 'Median_dupl_rate',
 'Median_total_nr_frag_in_regions',
 'Median_frip',
 'Median_tss_enrichment',
 'total_nr_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes_in_regions',
 'n_barcodes_merged',
 'frac_barcodes_merged',
 'efficiency',
 'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrM',
 'chrX',
 'chrY',
 'nonstandard',
 'total_fragments',
 'n_dars__B_cell',
 'top_2000_dars_median_logfc__B_cell',
 'top_2000_dars_median_fc__B_cell',
 'n_dars__CD14+_monocyte',
 'top_2000_dars_median_logfc__CD14+

In [51]:
if not "B_cells_bot20peaks_recovery" in df_stats.columns:
    df_stats = pd.concat([df_stats, df], axis=1)

# add median peak and dar tss distances

In [52]:
glob.glob("../fixedcells_3_cistopic_consensus/*median*tss_dist.tsv")

['../fixedcells_3_cistopic_consensus/median_dar_tss_dist.tsv',
 '../fixedcells_3_cistopic_consensus/median_frag_len_median_tss_dist.tsv',
 '../fixedcells_3_cistopic_consensus/median_peak_tss_dist.tsv',
 '../fixedcells_3_cistopic_consensus/median_top2kdar_tss_dist.tsv']

In [53]:
path = "../fixedcells_3_cistopic_consensus/median_dar_tss_dist.tsv"
df = pd.read_csv(path, sep="\t", index_col=0)
df.index = [x.split(".")[0] for x in df.index]
# df.index = [
#     x.replace("CNA_10xv11_4", "CNA_10xv11c_1")
#     .replace("CNA_10xv11_5", "CNA_10xv11c_2")
#     .replace("BRO_mtscatac", "BRO_mtscatacfacs")
#     for x in df.index
# ]

df_stats["alldars_median_dar_logfc"] = df["score"]
df_stats["alldars_median_dar_tss_dist"] = df["tss_dist"]

In [54]:
path = "../fixedcells_3_cistopic_consensus/median_peak_tss_dist.tsv"
df = pd.read_csv(path, sep="\t", index_col=0)
df.index = [x.split(".")[0] for x in df.index]
# df.index = [
#     x.replace("CNA_10xv11_4", "CNA_10xv11c_1")
#     .replace("CNA_10xv11_5", "CNA_10xv11c_2")
#     .replace("BRO_mtscatac", "BRO_mtscatacfacs")
#     for x in df.index
# ]
df_stats["allpeaks_median_peak_logfc"] = df["median_peak_score"]
df_stats["allpeaks_median_peak_tss_dist"] = df["median_peak_tss_dist"]

In [55]:
path = "../fixedcells_3_cistopic_consensus/median_top2kdar_tss_dist.tsv"
df = pd.read_csv(path, sep="\t", index_col=0)
df.index = [x.split(".")[0] for x in df.index]
# df.index = [
#     x.replace("CNA_10xv11_4", "CNA_10xv11c_1")
#     .replace("CNA_10xv11_5", "CNA_10xv11c_2")
#     .replace("BRO_mtscatac", "BRO_mtscatacfacs")
#     for x in df.index
# ]
df_stats["top2kdars_median_dar_logfc"] = df["score"]
df_stats["top2kdars_median_dar_tss_dist"] = df["tss_dist"]

# add median frag dist and len

In [56]:
path = "../fixedcells_3_cistopic_consensus/median_frag_len_median_tss_dist.tsv"
df = pd.read_csv(path, sep="\t", index_col=0)
df.index = [x.split(".")[0] for x in df.index]

In [57]:
if not "median_frag_len" in df_stats.columns:
    df_stats = pd.concat([df_stats, df], axis=1)

# add frag len quantiles

In [58]:
path = (
    "../fixedcells_3_cistopic_consensus/median_frag_len_median_tss_dist_quantiles.tsv"
)
df = pd.read_csv(path, sep="\t", index_col=0)
df.index = [x.split(".")[0] for x in df.index]

In [59]:
if not "mononucleosomal_distal" in df_stats.columns:
    df_stats = pd.concat([df_stats, df], axis=1)

# add fmx 

In [146]:
df = pd.read_csv(
    "../fixedcells_3_cistopic_consensus/out_fmx/genotype_concordance_unified.txt",
    sep="\t",
)
df

,INT_ID,BARCODE,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL,ubarcode,replicate,sample
CNA_hydrop_2.FIXEDCELLS.1,0,CGACATTACATAGGAGTCAA,182,182,SNG,"1,1",-324.11,"1,0",-343.05,18.94,0.00000,1.0,1,-324.11,0,-373.28,1.0,"1,0",-343.05,18.94,CNA_hydrop_2.FIXEDCELLS#CGACATTACATAGGAGTCAA,CNA_hydrop_2.FIXEDCELLS,sampleB
CNA_hydrop_2.FIXEDCELLS.2,1,GGCAACCTCTGAGCTAGTAA,209,209,SNG,"0,0",-369.10,"1,0",-390.29,21.19,0.00000,1.0,0,-369.10,1,-426.11,1.0,"1,0",-390.29,21.19,CNA_hydrop_2.FIXEDCELLS#GGCAACCTCTGAGCTAGTAA,CNA_hydrop_2.FIXEDCELLS,sampleA
CNA_hydrop_2.FIXEDCELLS.3,2,CAACACCATTTCTCGCACGA,273,273,SNG,"0,0",-492.20,"1,0",-520.48,28.28,0.00000,1.0,0,-492.20,1,-576.63,1.0,"1,0",-520.48,28.28,CNA_hydrop_2.FIXEDCELLS#CAACACCATTTCTCGCACGA,CNA_hydrop_2.FIXEDCELLS,sampleA
CNA_hydrop_2.FIXEDCELLS.4,3,TGCATGAGGTTACGGACGGT,354,354,SNG,"1,1",-636.69,"1,0",-669.83,33.14,0.00000,1.0,1,-636.69,0,-734.25,1.0,"1,0",-669.83,33.14,CNA_hydrop_2.FIXEDCELLS#TGCATGAGGTTACGGACGGT,CNA_hydrop_2.FIXEDCELLS,sampleB
CNA_hydrop_2.FIXEDCELLS.5,4,ACAGTGAAGATCCAGTGTTC,577,577,SNG,"0,0",-1069.99,"1,0",-1093.26,23.27,0.00000,1.0,0,-1069.99,1,-1173.29,1.0,"1,0",-1093.26,23.27,CNA_hydrop_2.FIXEDCELLS#ACAGTGAAGATCCAGTGTTC,CNA_hydrop_2.FIXEDCELLS,sampleA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIB_hydrop_12.FIXEDCELLS.1417,1416,GTCGTTGAGAGTGACCAGTA,126,126,SNG,"1,1",-221.86,"1,0",-240.94,19.08,0.00000,1.0,1,-221.86,0,-283.50,1.0,"1,0",-240.94,19.08,VIB_hydrop_12.FIXEDCELLS#GTCGTTGAGAGTGACCAGTA,VIB_hydrop_12.FIXEDCELLS,sampleB
VIB_hydrop_12.FIXEDCELLS.1418,1417,ACCGAAGGCTTTGCAGTTCT,103,103,SNG,"1,1",-185.75,"1,0",-198.81,13.06,0.00000,1.0,1,-185.75,0,-226.20,1.0,"1,0",-198.81,13.06,VIB_hydrop_12.FIXEDCELLS#ACCGAAGGCTTTGCAGTTCT,VIB_hydrop_12.FIXEDCELLS,sampleB
VIB_hydrop_12.FIXEDCELLS.1419,1418,TAGAGCCTGATTGTGTAGGA,115,115,SNG,"1,1",-207.21,"1,0",-219.27,12.06,-0.00001,1.0,1,-207.21,0,-244.26,1.0,"1,0",-219.27,12.06,VIB_hydrop_12.FIXEDCELLS#TAGAGCCTGATTGTGTAGGA,VIB_hydrop_12.FIXEDCELLS,sampleB
VIB_hydrop_12.FIXEDCELLS.1420,1419,CAATTGGAGACCACACGGAT,101,101,SNG,"1,1",-177.98,"1,0",-188.29,10.31,-0.00007,1.0,1,-177.98,0,-214.34,1.0,"1,0",-188.29,10.31,VIB_hydrop_12.FIXEDCELLS#CAATTGGAGACCACACGGAT,VIB_hydrop_12.FIXEDCELLS,sampleB


In [148]:
list(df.columns)

['INT_ID',
 'BARCODE',
 'NUM.SNPS',
 'NUM.READS',
 'DROPLET.TYPE',
 'BEST.GUESS',
 'BEST.LLK',
 'NEXT.GUESS',
 'NEXT.LLK',
 'DIFF.LLK.BEST.NEXT',
 'BEST.POSTERIOR',
 'SNG.POSTERIOR',
 'SNG.BEST.GUESS',
 'SNG.BEST.LLK',
 'SNG.NEXT.GUESS',
 'SNG.NEXT.LLK',
 'SNG.ONLY.POSTERIOR',
 'DBL.BEST.GUESS',
 'DBL.BEST.LLK',
 'DIFF.LLK.SNG.DBL',
 'ubarcode',
 'replicate',
 'sample']

In [153]:
df["tech"] = [x.split("_")[1] for x in df.index]

In [154]:
df.groupby("tech")["DIFF.LLK.SNG.DBL"].median()

tech
10xmultiome    143.02
10xv1          172.32
10xv11         157.45
10xv2          223.55
ddseq          141.21
hydrop          41.73
mtscatac       286.06
s3atac         215.79
Name: DIFF.LLK.SNG.DBL, dtype: float64

In [155]:
df.groupby("tech")["DIFF.LLK.BEST.NEXT"].median()

tech
10xmultiome    152.460
10xv1          175.975
10xv11         159.710
10xv2          234.100
ddseq          143.520
hydrop          42.150
mtscatac       288.780
s3atac         215.790
Name: DIFF.LLK.BEST.NEXT, dtype: float64

In [61]:
df_median = df.groupby("replicate").median()
df_median.index = [x.split(".")[0] for x in df_median.index]

/tmp/ipykernel_911774/560007370.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_median = df.groupby("replicate").median()


In [62]:
df_stats["fmx_n_snps"] = df_median["NUM.SNPS"]
df_stats["fmx_best_llk"] = df_median["BEST.LLK"]

# add male/female T cell ratios

In [63]:
df = pd.read_csv(
    "../fixedcells_3_cistopic_consensus/fraction_cd4_to_cd8_t_cells.tsv",
    sep="\t",
    index_col=0,
)
df.index = [x.split(".")[0] for x in df.index]

In [64]:
df_stats["ratio_cd4T_to_cd8T_in_male"] = df["fraction in male"]
df_stats["ratio_cd4T_to_cd8T_in_female"] = df["fraction in female"]
df_stats["ratio_cd4T_to_cd8T_normalized"] = df["fraction in female normalized"]

# add doublet counts

In [65]:
common = glob.glob(
    "../fixedcells_3_cistopic_consensus/cistopic_objects/*common_doublets.txt*"
)
scr_only = glob.glob(
    "../fixedcells_3_cistopic_consensus/cistopic_objects/*scr_doublets_unique.txt*"
)
fmx_only = glob.glob(
    "../fixedcells_3_cistopic_consensus/cistopic_objects/*fmx_doublets_unique.txt*"
)

In [66]:
for path in common:
    n_lines = sum(1 for line in open(path))
    # print(n_lines)
    sample = path.split("/")[-1].split(".")[0]
    df_stats.at[sample, "common_doublets"] = n_lines

df_stats["common_doublets"].replace(0, np.nan, inplace=True)

for path in scr_only:
    n_lines = sum(1 for line in open(path))
    # print(n_lines)
    sample = path.split("/")[-1].split(".")[0]
    df_stats.at[sample, "scr_exclusive_doublets"] = n_lines

for path in fmx_only:
    n_lines = sum(1 for line in open(path))
    # print(n_lines)
    sample = path.split("/")[-1].split(".")[0]
    df_stats.at[sample, "fmx_exclusive_doublets"] = n_lines

In [67]:
for var in [
    "fmx_n_snps",
    "scr_exclusive_doublets",
    "fmx_exclusive_doublets",
    "common_doublets",
]:
    df_stats.at["VIB_hydrop_1", var] = (
        df_stats.at["VIB_hydrop_11", var] + df_stats.at["VIB_hydrop_12", var]
    )
    df_stats.at["VIB_hydrop_2", var] = (
        df_stats.at["VIB_hydrop_21", var] + df_stats.at["VIB_hydrop_22", var]
    )

In [68]:
df_stats["total_doublets"] = (
    df_stats["scr_exclusive_doublets"]
    + df_stats["fmx_exclusive_doublets"]
    - df_stats["common_doublets"]
)

df_stats["total_doublets_pct"] = df_stats["total_doublets"] / df_stats["cells"]
df_stats["scr_exclusive_doublets_pct"] = (
    df_stats["scr_exclusive_doublets"] / df_stats["cells"]
)
df_stats["fmx_exclusive_doublets_pct"] = (
    df_stats["fmx_exclusive_doublets"] / df_stats["cells"]
)
df_stats["common_doublets_pct_of_doublets"] = (
    df_stats["common_doublets"] / df_stats["total_doublets"]
)
df_stats["common_doublets_pct"] = df_stats["common_doublets"] / df_stats["cells"]

# add cell counts

In [69]:
path_dict = {
    x.split("/")[-1].split(".")[0]: x
    for x in sorted(
        glob.glob(
            "../fixedcells_3_cistopic_consensus/cistopic_objects/*consensus.cell_data.tsv"
        )
    )
}
if verbose:
    print(path_dict)

In [70]:
df_merged = pd.DataFrame(index=df_stats.index)
for sample, path in path_dict.items():
    print(sample)
    df = pd.read_csv(path, index_col=0, sep="\t")
    df_stats.at[sample, "seurat_score"] = df["seurat_cell_type_pred_score"].median()
    df_sub = pd.DataFrame(df["seurat_cell_type"].value_counts())

    for cell_type in df_sub.index:
        col_name = "n_seurat_cells__" + cell_type
        df_merged.at[sample, col_name] = df_sub.loc[cell_type][0]

    df_sub = pd.DataFrame(df["seurat_cell_type"].value_counts())
    for cell_type in df_sub.index:
        col_name = "n_consensus_cells__" + cell_type
        df_merged.at[sample, col_name] = df_sub.loc[cell_type][0]

BIO_ddseq_1
BIO_ddseq_2
BIO_ddseq_3
BIO_ddseq_4
BRO_mtscatac_1
BRO_mtscatac_2
CNA_10xmultiome_1
CNA_10xmultiome_2
CNA_10xv11_1
CNA_10xv11_2
CNA_10xv11_3
CNA_10xv11_4
CNA_10xv11_5
CNA_10xv2_1
CNA_10xv2_2
CNA_hydrop_1
CNA_hydrop_2
CNA_hydrop_3
CNA_mtscatac_1
CNA_mtscatac_2
EPF_hydrop_1
EPF_hydrop_2
EPF_hydrop_3
EPF_hydrop_4
HAR_ddseq_1
HAR_ddseq_2
MDC_mtscatac_1
MDC_mtscatac_2
OHS_s3atac_1
OHS_s3atac_2
SAN_10xmultiome_1
SAN_10xmultiome_2
STA_10xv11_1
STA_10xv11_2
TXG_10xv11_1
TXG_10xv2_1
TXG_10xv2_2
UCS_ddseq_1
UCS_ddseq_2
VIB_10xmultiome_1
VIB_10xmultiome_2
VIB_10xv1_1
VIB_10xv1_2
VIB_10xv2_1
VIB_10xv2_2
VIB_hydrop_1
VIB_hydrop_2


In [71]:
df.columns

Index(['cisTopic_log_nr_acc', 'cisTopic_log_nr_frag', 'cisTopic_nr_frag',
       'cisTopic_nr_acc', 'Log_total_nr_frag', 'Log_unique_nr_frag',
       'Total_nr_frag', 'Unique_nr_frag', 'Dupl_nr_frag', 'Dupl_rate',
       'Total_nr_frag_in_regions', 'Unique_nr_frag_in_regions', 'FRIP',
       'TSS_enrichment', 'sample_id', 'barcode', 'Doublet_scores_fragments',
       'Predicted_doublets_fragments', 'fmx_droplet_type', 'fmx_sample',
       'seurat_cell_type', 'seurat_cell_type_pred_score',
       'pycisTopic_leiden_10_3.0', 'consensus_cell_type', 'UMAP_1', 'UMAP_2',
       'tSNE_1', 'tSNE_2'],
      dtype='object')

In [72]:
df_merged_rownorm = df_merged.copy()
df_merged_rownorm = df_merged_rownorm.div(df_merged_rownorm.sum(axis=1) / 2, axis=0)
df_merged_rownorm.columns = [
    x.replace(" ", "_").replace("n_", "pct_") for x in df_merged_rownorm.columns
]

In [73]:
if not "n_seurat_cells__CD4+ T cell" in df_stats.columns:
    df_stats = pd.concat([df_stats, df_merged], axis=1)


if not "pct_consensus_cells__CD4+_T_cell" in df_stats.columns:
    df_stats = pd.concat([df_stats, df_merged_rownorm], axis=1)

In [74]:
df_merged = pd.DataFrame(index=df_stats.index)
for sample, path in path_dict.items():
    print(sample)
    df = pd.read_csv(path, index_col=0, sep="\t")
    df_sub = pd.DataFrame(df["seurat_cell_type"].value_counts())

    df_merged.at[sample, "Median_Unique_nr_frag_in_regions"] = df[
        "Unique_nr_frag_in_regions"
    ].median()
    df_merged.at[sample, "Median_scrublet_doublet_scores_fragments"] = df[
        "Doublet_scores_fragments"
    ].median()

    df_merged.at[sample, "Mean_Unique_nr_frag_in_regions"] = df[
        "Unique_nr_frag_in_regions"
    ].mean()
    df_merged.at[sample, "Mean_scrublet_doublet_scores_fragments"] = df[
        "Doublet_scores_fragments"
    ].mean()

BIO_ddseq_1
BIO_ddseq_2
BIO_ddseq_3
BIO_ddseq_4
BRO_mtscatac_1
BRO_mtscatac_2
CNA_10xmultiome_1
CNA_10xmultiome_2
CNA_10xv11_1
CNA_10xv11_2
CNA_10xv11_3
CNA_10xv11_4
CNA_10xv11_5
CNA_10xv2_1
CNA_10xv2_2
CNA_hydrop_1
CNA_hydrop_2
CNA_hydrop_3
CNA_mtscatac_1
CNA_mtscatac_2
EPF_hydrop_1
EPF_hydrop_2
EPF_hydrop_3
EPF_hydrop_4
HAR_ddseq_1
HAR_ddseq_2
MDC_mtscatac_1
MDC_mtscatac_2
OHS_s3atac_1
OHS_s3atac_2
SAN_10xmultiome_1
SAN_10xmultiome_2
STA_10xv11_1
STA_10xv11_2
TXG_10xv11_1
TXG_10xv2_1
TXG_10xv2_2
UCS_ddseq_1
UCS_ddseq_2
VIB_10xmultiome_1
VIB_10xmultiome_2
VIB_10xv1_1
VIB_10xv1_2
VIB_10xv2_1
VIB_10xv2_2
VIB_hydrop_1
VIB_hydrop_2


In [75]:
df_merged["log_median_unique_nr_frag_in_regions"] = np.log10(
    df_merged["Median_Unique_nr_frag_in_regions"]
)

In [76]:
df_merged

,Median_Unique_nr_frag_in_regions,Median_scrublet_doublet_scores_fragments,Mean_Unique_nr_frag_in_regions,Mean_scrublet_doublet_scores_fragments,log_median_unique_nr_frag_in_regions
BIO_ddseq_1,3849.0,0.220779,3787.501305,0.216256,3.585348
BIO_ddseq_2,4972.0,0.190578,4808.578050,0.194312,3.696531
BIO_ddseq_3,4122.0,0.127753,4262.026468,0.137995,3.615108
BIO_ddseq_4,4611.0,0.127753,4752.200614,0.143042,3.663795
BRO_mtscatac_1,7814.0,0.076283,8514.084109,0.094029,3.892873
BRO_mtscatac_2,7623.0,0.086154,8207.707206,0.100562,3.882126
CNA_10xmultiome_1,5558.0,0.102990,6186.908235,0.120977,3.744919
CNA_10xmultiome_2,3784.0,0.132075,6054.972598,0.144116,3.577951
CNA_10xv11_1,4584.0,0.142857,5097.934648,0.150930,3.661245
CNA_10xv11_2,4951.0,0.114754,5439.018889,0.135385,3.694693


In [77]:
if not "Median_Unique_nr_frag_in_regions" in df_stats.columns:
    df_stats = pd.concat([df_stats, df_merged], axis=1)

In [78]:
df_stats.to_csv("fixedcells_general_statistics.tsv", sep="\t")

# calculate fric barplots

total reads > total fragments > in cells > unique > in peaks

In [79]:
df_sub = df_stats[
    [
        "reads",
        "total_nr_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes_in_regions",
    ]
]

In [80]:
df_sub["with_correct_barcode"] = df_sub["reads"] * df_stats["%_correct_barcodes"] / 100

/tmp/ipykernel_911774/24858831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["with_correct_barcode"] = df_sub["reads"] * df_stats["%_correct_barcodes"] / 100


In [81]:
df_sub["mapped"] = df_sub["with_correct_barcode"] * df_stats["%_mapq30"] / 100

/tmp/ipykernel_911774/3112989109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["mapped"] = df_sub["with_correct_barcode"] * df_stats["%_mapq30"] / 100


In [82]:
df_sub = df_sub.div(df_sub["reads"], axis=0)

In [83]:
df_sub["reads"] = df_sub["reads"] - df_sub["with_correct_barcode"]
df_sub["with_correct_barcode"] = df_sub["with_correct_barcode"] - df_sub["mapped"]
df_sub["mapped"] = df_sub["mapped"] - df_sub["total_nr_frag_in_selected_barcodes"]
df_sub["total_nr_frag_in_selected_barcodes"] = (
    df_sub["total_nr_frag_in_selected_barcodes"]
    - df_sub["total_nr_unique_frag_in_selected_barcodes"]
)
df_sub["total_nr_unique_frag_in_selected_barcodes"] = (
    df_sub["total_nr_unique_frag_in_selected_barcodes"]
    - df_sub["total_nr_unique_frag_in_selected_barcodes_in_regions"]
)

In [84]:
df_sub.columns = [
    "No correct barcode",
    "Duplicate fragments in cells",
    "Unique fragments in cells, not in peaks",
    "Unique fragments in cells and in peaks",
    "Not mapped properly",
    "Fragments in background noise barcodes",
]

In [85]:
df_sub = df_sub[
    [
        "No correct barcode",
        "Not mapped properly",
        "Fragments in background noise barcodes",
        "Duplicate fragments in cells",
        "Unique fragments in cells, not in peaks",
        "Unique fragments in cells and in peaks",
    ]
]

## add normalized

In [86]:
df_sub2 = df_stats[
    [
        "total_nr_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes",
        "total_nr_unique_frag_in_selected_barcodes_in_regions",
    ]
]

In [87]:
# df_sub2 = df_sub2.div(df_sub2["total_nr_frag_in_selected_barcodes"], axis=0)

In [88]:
df_sub2["Duplicate fragments in cells, normalized to fragments in cells"] = (
    df_sub2["total_nr_frag_in_selected_barcodes"]
    - df_sub2["total_nr_unique_frag_in_selected_barcodes"]
) / df_sub2["total_nr_frag_in_selected_barcodes"]


df_sub2[
    "Unique fragments in cells and in peaks, normalized to unique fragments in cells"
] = (
    df_sub2["total_nr_unique_frag_in_selected_barcodes_in_regions"]
    / df_sub2["total_nr_unique_frag_in_selected_barcodes"]
)

df_sub2[
    "Unique fragments in cells, not in peaks, normalized to unique fragments in cells"
] = (
    df_sub2["total_nr_unique_frag_in_selected_barcodes"]
    - df_sub2["total_nr_unique_frag_in_selected_barcodes_in_regions"]
) / df_sub2[
    "total_nr_unique_frag_in_selected_barcodes"
]

df_sub2 = df_sub2[
    [
        "Duplicate fragments in cells, normalized to fragments in cells",
        "Unique fragments in cells, not in peaks, normalized to unique fragments in cells",
        "Unique fragments in cells and in peaks, normalized to unique fragments in cells",
    ]
]

/tmp/ipykernel_911774/3097911818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub2["Duplicate fragments in cells, normalized to fragments in cells"] = (
/tmp/ipykernel_911774/3097911818.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub2[
/tmp/ipykernel_911774/3097911818.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [89]:
df_sub2

,"Duplicate fragments in cells, normalized to fragments in cells","Unique fragments in cells, not in peaks, normalized to unique fragments in cells","Unique fragments in cells and in peaks, normalized to unique fragments in cells"
BIO_ddseq_1,0.704205,0.450440,0.549560
BIO_ddseq_2,0.724223,0.378340,0.621660
BIO_ddseq_3,0.667613,0.371563,0.628437
BIO_ddseq_4,0.643843,0.370333,0.629667
BRO_mtscatac_1,0.442510,0.510438,0.489562
BRO_mtscatac_2,0.454560,0.516627,0.483373
CNA_10xmultiome_1,0.461778,0.411665,0.588335
CNA_10xmultiome_2,0.291976,0.473382,0.526618
CNA_10xv11_1,0.481966,0.381358,0.618642
CNA_10xv11_2,0.503942,0.360916,0.639084


In [90]:
df_sub = pd.concat([df_sub, df_sub2], axis=1)

In [91]:
df_sub

,No correct barcode,Not mapped properly,Fragments in background noise barcodes,Duplicate fragments in cells,"Unique fragments in cells, not in peaks",Unique fragments in cells and in peaks,"Duplicate fragments in cells, normalized to fragments in cells","Unique fragments in cells, not in peaks, normalized to unique fragments in cells","Unique fragments in cells and in peaks, normalized to unique fragments in cells"
BIO_ddseq_1,0.034600,0.091713,0.259337,0.432628,0.081855,0.099867,0.704205,0.450440,0.549560
BIO_ddseq_2,0.032400,0.076053,0.177040,0.517462,0.074550,0.122495,0.724223,0.378340,0.621660
BIO_ddseq_3,0.125300,0.111874,0.256717,0.337885,0.062506,0.105718,0.667613,0.371563,0.628437
BIO_ddseq_4,0.119000,0.112328,0.246076,0.336470,0.068929,0.117198,0.643843,0.370333,0.629667
BRO_mtscatac_1,0.053400,0.117852,0.059414,0.340438,0.218925,0.209971,0.442510,0.510438,0.489562
BRO_mtscatac_2,0.052100,0.116971,0.054692,0.352846,0.218735,0.204656,0.454560,0.516627,0.483373
CNA_10xmultiome_1,0.016900,0.101751,0.366111,0.237926,0.114160,0.163153,0.461778,0.411665,0.588335
CNA_10xmultiome_2,0.016000,0.104698,0.461293,0.122049,0.140102,0.155858,0.291976,0.473382,0.526618
CNA_10xv11_1,0.018100,0.097895,0.474439,0.197397,0.080912,0.131257,0.481966,0.381358,0.618642
CNA_10xv11_2,0.018300,0.097188,0.443402,0.222294,0.078974,0.139842,0.503942,0.360916,0.639084


In [92]:
df_sub.to_csv("fixedcells_general_losses.csv")

In [93]:
if not "No correct barcode" in df_stats.columns:
    df_stats = pd.concat([df_stats, df_sub], axis=1)

# save

In [94]:
cols = [x for x in list(df_stats.columns) if ".1" not in x]

In [95]:
tech_dict = {
    "10xmultiome": "10x Multiome",
    "10xv1": "10x v1",
    "10xv11": "10x v1.1",
    "10xv2": "10x v2",
    "ddseq": "ddSEQ SureCell",
    "hydrop": "HyDrop",
    "mtscatac": "mtscATAC-seq",
    "s3atac": "s3-ATAC",
}

tech_alias_dict = {
    "10xmultiome": "10x Multiome",
    "10xv1": "10x v1",
    "10xv11": "10x v1.1",
    "10xv11c": "10x v1.1 (control)",
    "10xv2": "10x v2",
    "ddseq": "Bio-Rad ddSEQ SureCell",
    "hydrop": "HyDrop",
    "mtscatac": "mtscATAC-seq",
    "mtscatacfacs": "mtscATAC-seq (FACS)",
    "s3atac": "s3-ATAC",
}

In [96]:
df_stats["centre"] = [
    centre_dict[x] for x in np.array([x.split("_") for x in list(df_stats.index)])[:, 0]
]

In [97]:
df_stats.index = [
    x.replace("CNA_10xv11_4", "CNA_10xv11c_1")
    .replace("CNA_10xv11_5", "CNA_10xv11c_2")
    .replace("BRO_mtscatac_1", "BRO_mtscatacfacs_1")
    .replace("BRO_mtscatac_2", "BRO_mtscatacfacs_2")
    for x in df_stats.index
]

In [98]:
df_stats["technology"] = [
    tech_alias_dict[x]
    for x in np.array([x.split("_") for x in list(df_stats.index)])[:, 1]
]

# check some na

In [99]:
df_stats = df_stats.drop(
    ["VIB_hydrop_11", "VIB_hydrop_12", "VIB_hydrop_21", "VIB_hydrop_22"]
)

In [100]:
pd.DataFrame(df_stats.isna().sum(axis=0))[
    pd.DataFrame(df_stats.isna().sum(axis=0))[0] > 0
]

,0
n_dars__B_cell,2
top_2000_dars_median_logfc__B_cell,2
top_2000_dars_median_fc__B_cell,2
n_dars__CD4+_T_cell,1
top_2000_dars_median_logfc__CD4+_T_cell,1
...,...
common_doublets_pct,21
n_seurat_cells__CD16+ monocyte,1
n_consensus_cells__CD16+ monocyte,1
pct_seurat_cells__CD16+_monocyte,1


# save

In [101]:
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [102]:
df_stats

,short_identifier,centre,technology,sequencing_instrument,reads,cells,RPC,nreads,%_correct_barcodes,r1_length,r2_length,avg_insert_size,%_mapq30,avg_map_quality,Median_total_nr_frag,Median_unique_nr_frag,Median_dupl_rate,Median_total_nr_frag_in_regions,Median_frip,Median_tss_enrichment,total_nr_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes_in_regions,n_barcodes_merged,frac_barcodes_merged,efficiency,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr2,chr20,chr21,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chrM,chrX,chrY,nonstandard,total_fragments,n_dars__B_cell,top_2000_dars_median_logfc__B_cell,top_2000_dars_median_fc__B_cell,n_dars__CD14+_monocyte,top_2000_dars_median_logfc__CD14+_monocyte,top_2000_dars_median_fc__CD14+_monocyte,n_dars__CD4+_T_cell,top_2000_dars_median_logfc__CD4+_T_cell,top_2000_dars_median_fc__CD4+_T_cell,n_dars__Cytotoxic_T_cell,top_2000_dars_median_logfc__Cytotoxic_T_cell,top_2000_dars_median_fc__Cytotoxic_T_cell,n_dars__Dendritic_cell,top_2000_dars_median_logfc__Dendritic_cell,top_2000_dars_median_fc__Dendritic_cell,n_dars__Natural_killer_cell,top_2000_dars_median_logfc__Natural_killer_cell,top_2000_dars_median_fc__Natural_killer_cell,n_dars__CD16+_monocyte,top_2000_dars_median_logfc__CD16+_monocyte,top_2000_dars_median_fc__CD16+_monocyte,n_peaks__Bcell,top10k_peaks_strength__Bcell,top10k_peaks_pvalBcell,top10k_peaks_qvalBcell,n_peaks__CD14_monocyte,top10k_peaks_strength__CD14_monocyte,top10k_peaks_pvalCD14_monocyte,top10k_peaks_qvalCD14_monocyte,n_peaks__CD4_Tcell,top10k_peaks_strength__CD4_Tcell,top10k_peaks_pvalCD4_Tcell,top10k_peaks_qvalCD4_Tcell,n_peaks__CytotoxicTcell,top10k_peaks_strength__CytotoxicTcell,top10k_peaks_pvalCytotoxicTcell,top10k_peaks_qvalCytotoxicTcell,n_peaks__Dendriticcell,top10k_peaks_strength__Dendriticcell,top10k_peaks_pvalDendriticcell,top10k_peaks_qvalDendriticcell,n_peaks__Naturalkillercell,top10k_peaks_strength__Naturalkillercell,top10k_peaks_pvalNaturalkillercell,top10k_peaks_qvalNaturalkillercell,n_peaks__CD16_monocyte,top10k_peaks_strength__CD16_monocyte,top10k_peaks_pvalCD16_monocyte,top10k_peaks_qvalCD16_monocyte,B_cells_bot20peaks_recovery,Naive_T_cells_bot20peaks_recovery,Cytotoxic_T_cells_bot20peaks_recovery,NK_cells_bot20peaks_recovery,CD14+_monocytes_bot20peaks_recovery,CD16+_monocytes_bot20peaks_recovery,Dendritic_cells_bot20peaks_recovery,mean_bot20peaks_recovery,B_cells_top20peaks_recovery,Naive_T_cells_top20peaks_recovery,Cytotoxic_T_cells_top20peaks_recovery,NK_cells_top20peaks_recovery,CD14+_monocytes_top20peaks_recovery,CD16+_monocytes_top20peaks_recovery,Dendritic_cells_top20peaks_recovery,mean_top20peaks_recovery,B_cell,CD14+_monocyte,CD16+_monocyte,CD4+_T_cell,Cytotoxic_T_cell,Dendritic_cell,Natural_killer_cell,mean_bot20dars_recovery,B_cell_top20dars_recovery,CD4+_T_cell_top20dars_recovery,Cytotoxic_T_cell_top20dars_recovery,Natural_killer_cell_top20dars_recovery,CD14+_monocyte_top20dars_recovery,CD16+_monocyte_top20dars_recovery,Dendritic_cell_top20dars_recovery,mean_top20dars_recovery,alldars_median_dar_logfc,alldars_median_dar_tss_dist,allpeaks_median_peak_logfc,allpeaks_median_peak_tss_dist,top2kdars_median_dar_logfc,top2kdars_median_dar_tss_dist,median_frag_len,median_log10_frag_dist_nearest_tss,median_frag_dist_nearest_tss,nucleosome-free_proximal,mononucleosomal_distal,mononucleosomal_proximal,nucleosome-free_distal,multinucleosomal_distal,multinucleosomal_proximal,nucleosome-free,mononucleosomal,multinucleosomal,proximal,distal,fmx_n_snps,fmx_best_llk,ratio_cd4T_to_cd8T_in_male,ratio_cd4T_to_cd8T_in_female,ratio_cd4T_to_cd8T_normalized,common_doublets,scr_exclusive_doublets,fmx_exclusive_doublets,total_doublets,total_doublets_pct,scr_exclusive_doublets_pct,fmx_exclusive_doublets_pct,common_doublets_pct_of_doublets,common_doublets_pct,seurat_score,n_seurat_cells__CD4+ T cell,n_seurat_cells__Cytotoxic T cell,n_seurat_cells__CD14+ monocyte,n_seurat_cells__B cell,n_seur

In [103]:
df_stats[cols].to_csv("fixedcells_general_statistics.tsv", sep="\t")

# write smaller df to excel:

In [104]:
list(df_stats.columns)

['short_identifier',
 'centre',
 'technology',
 'sequencing_instrument',
 'reads',
 'cells',
 'RPC',
 'nreads',
 '%_correct_barcodes',
 'r1_length',
 'r2_length',
 'avg_insert_size',
 '%_mapq30',
 'avg_map_quality',
 'Median_total_nr_frag',
 'Median_unique_nr_frag',
 'Median_dupl_rate',
 'Median_total_nr_frag_in_regions',
 'Median_frip',
 'Median_tss_enrichment',
 'total_nr_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes_in_regions',
 'n_barcodes_merged',
 'frac_barcodes_merged',
 'efficiency',
 'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrM',
 'chrX',
 'chrY',
 'nonstandard',
 'total_fragments',
 'n_dars__B_cell',
 'top_2000_dars_median_logfc__B_cell',
 'top_2000_dars_median_fc__B_cell',
 'n_dars__CD14+_monocyte',
 'top_2000_dars_median_logfc__CD14+

In [105]:
cols_sorted = [
    "short_identifier",
    "centre",
    "technology",
    "sequencing_instrument",
    "reads",
    "cells",
    "RPC",
    "nreads",
    "%_correct_barcodes",
    "r1_length",
    "r2_length",
    "avg_insert_size",
    "%_mapq30",
    "avg_map_quality",
    "Median_total_nr_frag",
    "Median_unique_nr_frag",
    "Median_dupl_rate",
    "Median_total_nr_frag_in_regions",
    "Median_frip",
    "Median_tss_enrichment",
    "total_nr_frag_in_selected_barcodes",
    "total_nr_unique_frag_in_selected_barcodes",
    "total_nr_unique_frag_in_selected_barcodes_in_regions",
    "n_barcodes_merged",
    "frac_barcodes_merged",
    "No correct barcode",
    "Not mapped properly",
    "Fragments in background noise barcodes",
    "Duplicate fragments in cells",
    "Unique fragments in cells, not in peaks",
    "Unique fragments in cells and in peaks",
    "Duplicate fragments in cells, normalized to fragments in cells",
    "Unique fragments in cells, not in peaks, normalized to unique fragments in cells",
    "Unique fragments in cells and in peaks, normalized to unique fragments in cells",
    "Mean_scrublet_doublet_scores_fragments",
    "scr_exclusive_doublets",
    "fmx_exclusive_doublets",
    "common_doublets",
    "total_doublets",
    "total_doublets_pct",
    "scr_exclusive_doublets_pct",
    "fmx_exclusive_doublets_pct",
    "common_doublets_pct_of_doublets",
    "common_doublets_pct",
    "Median_Unique_nr_frag_in_regions",
    "Median_scrublet_doublet_scores_fragments",
    "fmx_n_snps",
    "fmx_best_llk",
    "ratio_cd4T_to_cd8T_in_male",
    "ratio_cd4T_to_cd8T_in_female",
    "ratio_cd4T_to_cd8T_normalized",
    "total_fragments",
    "chr1",
    "chr10",
    "chr11",
    "chr12",
    "chr13",
    "chr14",
    "chr15",
    "chr16",
    "chr17",
    "chr18",
    "chr19",
    "chr2",
    "chr20",
    "chr21",
    "chr22",
    "chr3",
    "chr4",
    "chr5",
    "chr6",
    "chr7",
    "chr8",
    "chr9",
    "chrM",
    "chrX",
    "chrY",
    "nonstandard",
    "nucleosome-free_proximal",
    "mononucleosomal_distal",
    "mononucleosomal_proximal",
    "nucleosome-free_distal",
    "multinucleosomal_distal",
    "multinucleosomal_proximal",
    "nucleosome-free",
    "mononucleosomal",
    "multinucleosomal",
    "proximal",
    "distal",
    "seurat_score",
    "n_seurat_cells__CD4+ T cell",
    "n_seurat_cells__Cytotoxic T cell",
    "n_seurat_cells__CD14+ monocyte",
    "n_seurat_cells__B cell",
    "n_seurat_cells__Natural killer cell",
    "n_seurat_cells__CD16+ monocyte",
    "n_seurat_cells__Dendritic cell",
    "n_consensus_cells__CD4+ T cell",
    "n_consensus_cells__Cytotoxic T cell",
    "n_consensus_cells__CD14+ monocyte",
    "n_consensus_cells__B cell",
    "n_consensus_cells__Natural killer cell",
    "n_consensus_cells__CD16+ monocyte",
    "n_consensus_cells__Dendritic cell",
    "pct_seurat_cells__CD4+_T_cell",
    "pct_seurat_cells__Cytotoxic_T_cell",
    "pct_seurat_cells__CD14+_monocyte",
    "pct_seurat_cells__B_cell",
    "pct_seurat_cells__Natural_killer_cell",
    "pct_seurat_cells__CD16+_monocyte",
    "pct_seurat_cells__Dendritic_cell",
    "pct_consensus_cells__CD4+_T_cell",
    "pct_consensus_cells__Cytotoxic_T_cell",
    "pct_consensus_cells__CD14+_monocyte",
    "pct_consensus_cells__B_cell",
    "pct_consensus_cells__Natural_killer_cell",
    "pct_consensus_cells__CD16+_monocyte",
    "pct_consensus_cells__Dendritic_cell",
    "alldars_median_dar_logfc",
    "alldars_median_dar_tss_dist",
    "allpeaks_median_peak_logfc",
    "allpeaks_median_peak_tss_dist",
    "top2kdars_median_dar_logfc",
    "top2kdars_median_dar_tss_dist",
    "median_frag_len",
    "median_log10_frag_dist_nearest_tss",
    "median_frag_dist_nearest_tss",
    "n_dars__B_cell",
    "top_2000_dars_median_logfc__B_cell",
    "top_2000_dars_median_fc__B_cell",
    "n_dars__CD14+_monocyte",
    "top_2000_dars_median_logfc__CD14+_monocyte",
    "top_2000_dars_median_fc__CD14+_monocyte",
    "n_dars__CD4+_T_cell",
    "top_2000_dars_median_logfc__CD4+_T_cell",
    "top_2000_dars_median_fc__CD4+_T_cell",
    "n_dars__Cytotoxic_T_cell",
    "top_2000_dars_median_logfc__Cytotoxic_T_cell",
    "top_2000_dars_median_fc__Cytotoxic_T_cell",
    "n_dars__Dendritic_cell",
    "top_2000_dars_median_logfc__Dendritic_cell",
    "top_2000_dars_median_fc__Dendritic_cell",
    "n_dars__Natural_killer_cell",
    "top_2000_dars_median_logfc__Natural_killer_cell",
    "top_2000_dars_median_fc__Natural_killer_cell",
    "n_dars__CD16+_monocyte",
    "top_2000_dars_median_logfc__CD16+_monocyte",
    "top_2000_dars_median_fc__CD16+_monocyte",
    "n_peaks__Bcell",
    "top10k_peaks_strength__Bcell",
    "n_peaks__CD14_monocyte",
    "top10k_peaks_strength__CD14_monocyte",
    "n_peaks__CD4_Tcell",
    "top10k_peaks_strength__CD4_Tcell",
    "n_peaks__CytotoxicTcell",
    "top10k_peaks_strength__CytotoxicTcell",
    "n_peaks__Dendriticcell",
    "top10k_peaks_strength__Dendriticcell",
    "n_peaks__Naturalkillercell",
    "top10k_peaks_strength__Naturalkillercell",
    "n_peaks__CD16_monocyte",
    "top10k_peaks_strength__CD16_monocyte",
]
rows_sorted = [
    "VIB_10xv1_1",
    "VIB_10xv1_2",
    "CNA_10xv11_1",
    "CNA_10xv11_2",
    "CNA_10xv11_3",
    "CNA_10xv11c_1",
    "CNA_10xv11c_2",
    "STA_10xv11_1",
    "STA_10xv11_2",
    "TXG_10xv11_1",
    "CNA_10xv2_1",
    "CNA_10xv2_2",
    "TXG_10xv2_1",
    "TXG_10xv2_2",
    "VIB_10xv2_1",
    "VIB_10xv2_2",
    "CNA_10xmultiome_1",
    "CNA_10xmultiome_2",
    "SAN_10xmultiome_1",
    "SAN_10xmultiome_2",
    "VIB_10xmultiome_1",
    "VIB_10xmultiome_2",
    "BRO_mtscatacfacs_1",
    "BRO_mtscatacfacs_2",
    "CNA_mtscatac_1",
    "CNA_mtscatac_2",
    "MDC_mtscatac_1",
    "MDC_mtscatac_2",
    "BIO_ddseq_1",
    "BIO_ddseq_2",
    "BIO_ddseq_3",
    "BIO_ddseq_4",
    "HAR_ddseq_1",
    "HAR_ddseq_2",
    "UCS_ddseq_1",
    "UCS_ddseq_2",
    "OHS_s3atac_1",
    "OHS_s3atac_2",
    "CNA_hydrop_1",
    "CNA_hydrop_2",
    "CNA_hydrop_3",
    "EPF_hydrop_1",
    "EPF_hydrop_2",
    "EPF_hydrop_3",
    "EPF_hydrop_4",
    "VIB_hydrop_1",
    "VIB_hydrop_2",
]

In [106]:
df_stats.loc[rows_sorted][cols_sorted].to_csv(
    "fixedcells_general_statistics_table.tsv", sep="\t", header=True
)

# check some things for text

In [107]:
df_stats = pd.read_csv("fixedcells_general_statistics.tsv", sep="\t", index_col=0)

In [108]:
df_stats.groupby("technology")["Median_frip"].median()

technology
10x Multiome              0.606648
10x v1                    0.574628
10x v1.1                  0.573633
10x v1.1 (control)        0.616573
10x v2                    0.619329
Bio-Rad ddSEQ SureCell    0.606599
HyDrop                    0.412306
mtscATAC-seq              0.390855
mtscATAC-seq (FACS)       0.495800
s3-ATAC                   0.189464
Name: Median_frip, dtype: float64

In [109]:
df_medians = df_stats.groupby("technology").median()
df_means = df_stats.groupby("technology").mean()

/tmp/ipykernel_911774/3686274921.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_medians = df_stats.groupby("technology").median()
/tmp/ipykernel_911774/3686274921.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_means = df_stats.groupby("technology").mean()


In [110]:
series = df_medians["No correct barcode"] + df_medians["Not mapped properly"]
series.sort_values()

technology
10x v2                    0.104695
10x Multiome              0.118274
10x v1.1                  0.120092
10x v1                    0.142169
mtscATAC-seq              0.142807
Bio-Rad ddSEQ SureCell    0.143326
10x v1.1 (control)        0.145505
s3-ATAC                   0.147050
mtscATAC-seq (FACS)       0.170161
HyDrop                    0.226784
dtype: float64

In [111]:
df_means["Fragments in background noise barcodes"].sort_values()

technology
mtscATAC-seq (FACS)       0.057053
10x v2                    0.068884
10x v1.1 (control)        0.178743
Bio-Rad ddSEQ SureCell    0.339764
mtscATAC-seq              0.361478
HyDrop                    0.396965
10x v1.1                  0.440424
10x Multiome              0.475981
10x v1                    0.493503
s3-ATAC                   0.624330
Name: Fragments in background noise barcodes, dtype: float64

In [112]:
df_means["Duplicate fragments in cells"].sort_values()

technology
s3-ATAC                   0.011772
10x v1                    0.090516
10x Multiome              0.121836
10x v1.1                  0.158550
mtscATAC-seq              0.189433
10x v1.1 (control)        0.260713
HyDrop                    0.289868
Bio-Rad ddSEQ SureCell    0.307599
mtscATAC-seq (FACS)       0.346642
10x v2                    0.376667
Name: Duplicate fragments in cells, dtype: float64

In [113]:
df_means["Duplicate fragments in cells, normalized to fragments in cells"].sort_values()

technology
s3-ATAC                   0.051093
10x v1                    0.247749
10x Multiome              0.314283
10x v1.1                  0.360317
mtscATAC-seq              0.368760
10x v1.1 (control)        0.385801
mtscATAC-seq (FACS)       0.448535
10x v2                    0.457507
Bio-Rad ddSEQ SureCell    0.579556
HyDrop                    0.716042
Name: Duplicate fragments in cells, normalized to fragments in cells, dtype: float64

In [114]:
df_means

,reads,cells,RPC,nreads,%_correct_barcodes,r1_length,r2_length,avg_insert_size,%_mapq30,avg_map_quality,Median_total_nr_frag,Median_unique_nr_frag,Median_dupl_rate,Median_total_nr_frag_in_regions,Median_frip,Median_tss_enrichment,total_nr_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes_in_regions,n_barcodes_merged,frac_barcodes_merged,efficiency,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr2,chr20,chr21,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chrM,chrX,chrY,nonstandard,total_fragments,n_dars__B_cell,top_2000_dars_median_logfc__B_cell,top_2000_dars_median_fc__B_cell,n_dars__CD14+_monocyte,top_2000_dars_median_logfc__CD14+_monocyte,top_2000_dars_median_fc__CD14+_monocyte,n_dars__CD4+_T_cell,top_2000_dars_median_logfc__CD4+_T_cell,top_2000_dars_median_fc__CD4+_T_cell,n_dars__Cytotoxic_T_cell,top_2000_dars_median_logfc__Cytotoxic_T_cell,top_2000_dars_median_fc__Cytotoxic_T_cell,n_dars__Dendritic_cell,top_2000_dars_median_logfc__Dendritic_cell,top_2000_dars_median_fc__Dendritic_cell,n_dars__Natural_killer_cell,top_2000_dars_median_logfc__Natural_killer_cell,top_2000_dars_median_fc__Natural_killer_cell,n_dars__CD16+_monocyte,top_2000_dars_median_logfc__CD16+_monocyte,top_2000_dars_median_fc__CD16+_monocyte,n_peaks__Bcell,top10k_peaks_strength__Bcell,top10k_peaks_pvalBcell,top10k_peaks_qvalBcell,n_peaks__CD14_monocyte,top10k_peaks_strength__CD14_monocyte,top10k_peaks_pvalCD14_monocyte,top10k_peaks_qvalCD14_monocyte,n_peaks__CD4_Tcell,top10k_peaks_strength__CD4_Tcell,top10k_peaks_pvalCD4_Tcell,top10k_peaks_qvalCD4_Tcell,n_peaks__CytotoxicTcell,top10k_peaks_strength__CytotoxicTcell,top10k_peaks_pvalCytotoxicTcell,top10k_peaks_qvalCytotoxicTcell,n_peaks__Dendriticcell,top10k_peaks_strength__Dendriticcell,top10k_peaks_pvalDendriticcell,top10k_peaks_qvalDendriticcell,n_peaks__Naturalkillercell,top10k_peaks_strength__Naturalkillercell,top10k_peaks_pvalNaturalkillercell,top10k_peaks_qvalNaturalkillercell,n_peaks__CD16_monocyte,top10k_peaks_strength__CD16_monocyte,top10k_peaks_pvalCD16_monocyte,top10k_peaks_qvalCD16_monocyte,B_cells_bot20peaks_recovery,Naive_T_cells_bot20peaks_recovery,Cytotoxic_T_cells_bot20peaks_recovery,NK_cells_bot20peaks_recovery,CD14+_monocytes_bot20peaks_recovery,CD16+_monocytes_bot20peaks_recovery,Dendritic_cells_bot20peaks_recovery,mean_bot20peaks_recovery,B_cells_top20peaks_recovery,Naive_T_cells_top20peaks_recovery,Cytotoxic_T_cells_top20peaks_recovery,NK_cells_top20peaks_recovery,CD14+_monocytes_top20peaks_recovery,CD16+_monocytes_top20peaks_recovery,Dendritic_cells_top20peaks_recovery,mean_top20peaks_recovery,B_cell,CD14+_monocyte,CD16+_monocyte,CD4+_T_cell,Cytotoxic_T_cell,Dendritic_cell,Natural_killer_cell,mean_bot20dars_recovery,B_cell_top20dars_recovery,CD4+_T_cell_top20dars_recovery,Cytotoxic_T_cell_top20dars_recovery,Natural_killer_cell_top20dars_recovery,CD14+_monocyte_top20dars_recovery,CD16+_monocyte_top20dars_recovery,Dendritic_cell_top20dars_recovery,mean_top20dars_recovery,alldars_median_dar_logfc,alldars_median_dar_tss_dist,allpeaks_median_peak_logfc,allpeaks_median_peak_tss_dist,top2kdars_median_dar_logfc,top2kdars_median_dar_tss_dist,median_frag_len,median_log10_frag_dist_nearest_tss,median_frag_dist_nearest_tss,nucleosome-free_proximal,mononucleosomal_distal,mononucleosomal_proximal,nucleosome-free_distal,multinucleosomal_distal,multinucleosomal_proximal,nucleosome-free,mononucleosomal,multinucleosomal,proximal,distal,fmx_n_snps,fmx_best_llk,ratio_cd4T_to_cd8T_in_male,ratio_cd4T_to_cd8T_in_female,ratio_cd4T_to_cd8T_normalized,common_doublets,scr_exclusive_doublets,fmx_exclusive_doublets,total_doublets,total_doublets_pct,scr_exclusive_doublets_pct,fmx_exclusive_doublets_pct,common_doublets_pct_of_doublets,common_doublets_pct,seurat_score,n_seurat_cells__CD4+ T cell,n_seurat_cells__Cytotoxic T cell,n_seurat_cells__CD14+ monocyte,n_seurat_cells__B cell,n_seurat_cells__Natural killer cell,n_seurat_cells__CD16+ monoc

In [115]:
df_means[
    "Unique fragments in cells, not in peaks, normalized to unique fragments in cells"
].sort_values()

technology
10x v2                    0.391402
10x Multiome              0.418831
10x v1.1 (control)        0.419502
Bio-Rad ddSEQ SureCell    0.429532
10x v1                    0.432239
10x v1.1                  0.435052
mtscATAC-seq (FACS)       0.513533
mtscATAC-seq              0.635342
HyDrop                    0.636084
s3-ATAC                   0.821242
Name: Unique fragments in cells, not in peaks, normalized to unique fragments in cells, dtype: float64

In [116]:
df_means

,reads,cells,RPC,nreads,%_correct_barcodes,r1_length,r2_length,avg_insert_size,%_mapq30,avg_map_quality,Median_total_nr_frag,Median_unique_nr_frag,Median_dupl_rate,Median_total_nr_frag_in_regions,Median_frip,Median_tss_enrichment,total_nr_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes,total_nr_unique_frag_in_selected_barcodes_in_regions,n_barcodes_merged,frac_barcodes_merged,efficiency,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr2,chr20,chr21,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chrM,chrX,chrY,nonstandard,total_fragments,n_dars__B_cell,top_2000_dars_median_logfc__B_cell,top_2000_dars_median_fc__B_cell,n_dars__CD14+_monocyte,top_2000_dars_median_logfc__CD14+_monocyte,top_2000_dars_median_fc__CD14+_monocyte,n_dars__CD4+_T_cell,top_2000_dars_median_logfc__CD4+_T_cell,top_2000_dars_median_fc__CD4+_T_cell,n_dars__Cytotoxic_T_cell,top_2000_dars_median_logfc__Cytotoxic_T_cell,top_2000_dars_median_fc__Cytotoxic_T_cell,n_dars__Dendritic_cell,top_2000_dars_median_logfc__Dendritic_cell,top_2000_dars_median_fc__Dendritic_cell,n_dars__Natural_killer_cell,top_2000_dars_median_logfc__Natural_killer_cell,top_2000_dars_median_fc__Natural_killer_cell,n_dars__CD16+_monocyte,top_2000_dars_median_logfc__CD16+_monocyte,top_2000_dars_median_fc__CD16+_monocyte,n_peaks__Bcell,top10k_peaks_strength__Bcell,top10k_peaks_pvalBcell,top10k_peaks_qvalBcell,n_peaks__CD14_monocyte,top10k_peaks_strength__CD14_monocyte,top10k_peaks_pvalCD14_monocyte,top10k_peaks_qvalCD14_monocyte,n_peaks__CD4_Tcell,top10k_peaks_strength__CD4_Tcell,top10k_peaks_pvalCD4_Tcell,top10k_peaks_qvalCD4_Tcell,n_peaks__CytotoxicTcell,top10k_peaks_strength__CytotoxicTcell,top10k_peaks_pvalCytotoxicTcell,top10k_peaks_qvalCytotoxicTcell,n_peaks__Dendriticcell,top10k_peaks_strength__Dendriticcell,top10k_peaks_pvalDendriticcell,top10k_peaks_qvalDendriticcell,n_peaks__Naturalkillercell,top10k_peaks_strength__Naturalkillercell,top10k_peaks_pvalNaturalkillercell,top10k_peaks_qvalNaturalkillercell,n_peaks__CD16_monocyte,top10k_peaks_strength__CD16_monocyte,top10k_peaks_pvalCD16_monocyte,top10k_peaks_qvalCD16_monocyte,B_cells_bot20peaks_recovery,Naive_T_cells_bot20peaks_recovery,Cytotoxic_T_cells_bot20peaks_recovery,NK_cells_bot20peaks_recovery,CD14+_monocytes_bot20peaks_recovery,CD16+_monocytes_bot20peaks_recovery,Dendritic_cells_bot20peaks_recovery,mean_bot20peaks_recovery,B_cells_top20peaks_recovery,Naive_T_cells_top20peaks_recovery,Cytotoxic_T_cells_top20peaks_recovery,NK_cells_top20peaks_recovery,CD14+_monocytes_top20peaks_recovery,CD16+_monocytes_top20peaks_recovery,Dendritic_cells_top20peaks_recovery,mean_top20peaks_recovery,B_cell,CD14+_monocyte,CD16+_monocyte,CD4+_T_cell,Cytotoxic_T_cell,Dendritic_cell,Natural_killer_cell,mean_bot20dars_recovery,B_cell_top20dars_recovery,CD4+_T_cell_top20dars_recovery,Cytotoxic_T_cell_top20dars_recovery,Natural_killer_cell_top20dars_recovery,CD14+_monocyte_top20dars_recovery,CD16+_monocyte_top20dars_recovery,Dendritic_cell_top20dars_recovery,mean_top20dars_recovery,alldars_median_dar_logfc,alldars_median_dar_tss_dist,allpeaks_median_peak_logfc,allpeaks_median_peak_tss_dist,top2kdars_median_dar_logfc,top2kdars_median_dar_tss_dist,median_frag_len,median_log10_frag_dist_nearest_tss,median_frag_dist_nearest_tss,nucleosome-free_proximal,mononucleosomal_distal,mononucleosomal_proximal,nucleosome-free_distal,multinucleosomal_distal,multinucleosomal_proximal,nucleosome-free,mononucleosomal,multinucleosomal,proximal,distal,fmx_n_snps,fmx_best_llk,ratio_cd4T_to_cd8T_in_male,ratio_cd4T_to_cd8T_in_female,ratio_cd4T_to_cd8T_normalized,common_doublets,scr_exclusive_doublets,fmx_exclusive_doublets,total_doublets,total_doublets_pct,scr_exclusive_doublets_pct,fmx_exclusive_doublets_pct,common_doublets_pct_of_doublets,common_doublets_pct,seurat_score,n_seurat_cells__CD4+ T cell,n_seurat_cells__Cytotoxic T cell,n_seurat_cells__CD14+ monocyte,n_seurat_cells__B cell,n_seurat_cells__Natural killer cell,n_seurat_cells__CD16+ monoc

In [117]:
df_means[
    "Unique fragments in cells and in peaks, normalized to unique fragments in cells"
].sort_values()

technology
s3-ATAC                   0.178758
HyDrop                    0.363916
mtscATAC-seq              0.364658
mtscATAC-seq (FACS)       0.486467
10x v1.1                  0.564948
10x v1                    0.567761
Bio-Rad ddSEQ SureCell    0.570468
10x v1.1 (control)        0.580498
10x Multiome              0.581169
10x v2                    0.608598
Name: Unique fragments in cells and in peaks, normalized to unique fragments in cells, dtype: float64

In [118]:
df_medians["Unique fragments in cells and in peaks"].sort_values()

technology
s3-ATAC                   0.041354
HyDrop                    0.043792
mtscATAC-seq              0.103005
Bio-Rad ddSEQ SureCell    0.111458
10x v1.1                  0.137735
10x v1                    0.153879
10x Multiome              0.159505
mtscATAC-seq (FACS)       0.207314
10x v1.1 (control)        0.240739
10x v2                    0.279082
Name: Unique fragments in cells and in peaks, dtype: float64

In [119]:
df_stats["n_peaks__Bcell"]

BIO_ddseq_1            57701.0
BIO_ddseq_2            62368.0
BIO_ddseq_3            59383.0
BIO_ddseq_4            62871.0
BRO_mtscatacfacs_1     71752.0
BRO_mtscatacfacs_2     58383.0
CNA_10xmultiome_1      74017.0
CNA_10xmultiome_2      56656.0
CNA_10xv11_1           57249.0
CNA_10xv11_2           52223.0
CNA_10xv11_3          109204.0
CNA_10xv11c_1          60050.0
CNA_10xv11c_2          93974.0
CNA_10xv2_1           130659.0
CNA_10xv2_2           145489.0
CNA_hydrop_1               NaN
CNA_hydrop_2           43706.0
CNA_hydrop_3            8785.0
CNA_mtscatac_1         53662.0
CNA_mtscatac_2         32664.0
EPF_hydrop_1           45063.0
EPF_hydrop_2           26942.0
EPF_hydrop_3           33573.0
EPF_hydrop_4           33886.0
HAR_ddseq_1            82924.0
HAR_ddseq_2            80476.0
MDC_mtscatac_1         97089.0
MDC_mtscatac_2         69003.0
OHS_s3atac_1           64256.0
OHS_s3atac_2           20100.0
SAN_10xmultiome_1      88741.0
SAN_10xmultiome_2     108238.0
STA_10xv

In [120]:
list(df_stats.columns)

['short_identifier',
 'centre',
 'technology',
 'sequencing_instrument',
 'reads',
 'cells',
 'RPC',
 'nreads',
 '%_correct_barcodes',
 'r1_length',
 'r2_length',
 'avg_insert_size',
 '%_mapq30',
 'avg_map_quality',
 'Median_total_nr_frag',
 'Median_unique_nr_frag',
 'Median_dupl_rate',
 'Median_total_nr_frag_in_regions',
 'Median_frip',
 'Median_tss_enrichment',
 'total_nr_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes',
 'total_nr_unique_frag_in_selected_barcodes_in_regions',
 'n_barcodes_merged',
 'frac_barcodes_merged',
 'efficiency',
 'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrM',
 'chrX',
 'chrY',
 'nonstandard',
 'total_fragments',
 'n_dars__B_cell',
 'top_2000_dars_median_logfc__B_cell',
 'top_2000_dars_median_fc__B_cell',
 'n_dars__CD14+_monocyte',
 'top_2000_dars_median_logfc__CD14+

In [138]:
cell_data_path_dict = {
    os.path.basename(x).split(".")[0]: x
    for x in sorted(
        glob.glob(
            "../fixedcells_3_cistopic_consensus/cistopic_objects/*dimreduc.consensus*cell_data.tsv"
        )
    )
}

In [143]:
df_merged = pd.DataFrame()
for sample, path in cell_data_path_dict.items():
    print(sample)
    df = pd.read_csv(path, sep="\t", index_col=0)

    df_merged = pd.concat([df_merged, df])

BIO_ddseq_1
BIO_ddseq_2
BIO_ddseq_3
BIO_ddseq_4
BRO_mtscatac_1
BRO_mtscatac_2
CNA_10xmultiome_1
CNA_10xmultiome_2
CNA_10xv11_1
CNA_10xv11_2
CNA_10xv11_3
CNA_10xv11_4
CNA_10xv11_5
CNA_10xv2_1
CNA_10xv2_2
CNA_hydrop_1
CNA_hydrop_2
CNA_hydrop_3
CNA_mtscatac_1
CNA_mtscatac_2
EPF_hydrop_1
EPF_hydrop_2
EPF_hydrop_3
EPF_hydrop_4
HAR_ddseq_1
HAR_ddseq_2
MDC_mtscatac_1
MDC_mtscatac_2
OHS_s3atac_1
OHS_s3atac_2
SAN_10xmultiome_1
SAN_10xmultiome_2
STA_10xv11_1
STA_10xv11_2
TXG_10xv11_1
TXG_10xv2_1
TXG_10xv2_2
UCS_ddseq_1
UCS_ddseq_2
VIB_10xmultiome_1
VIB_10xmultiome_2
VIB_10xv1_1
VIB_10xv1_2
VIB_10xv2_1
VIB_10xv2_2
VIB_hydrop_1
VIB_hydrop_2


,cisTopic_log_nr_acc,cisTopic_log_nr_frag,cisTopic_nr_frag,cisTopic_nr_acc,Log_total_nr_frag,Log_unique_nr_frag,Total_nr_frag,Unique_nr_frag,Dupl_nr_frag,Dupl_rate,Total_nr_frag_in_regions,Unique_nr_frag_in_regions,FRIP,TSS_enrichment,sample_id,barcode,Doublet_scores_fragments,Predicted_doublets_fragments,seurat_cell_type,seurat_cell_type_pred_score,pycisTopic_leiden_10_3.0,consensus_cell_type,UMAP_1,UMAP_2,tSNE_1,tSNE_2,fmx_droplet_type,fmx_sample
CAGGCGGATGAATAAAGTGCG_CGCGGCGACCTACCGCAGTGT___BIO_ddseq_1.FIXEDCELLS,3.684217,3.775756,5967,4833,4.495669,3.925415,31309,8422,22887,0.731004,16299,4660,0.553313,38.887733,BIO_ddseq_1.FIXEDCELLS,CAGGCGGATGAATAAAGTGCG_CGCGGCGACCTACCGCAGTGT,0.170213,False,CD4+ T cell,0.794101,18,CD4+ T cell,-1.676865,-1.709572,-22.865466,-16.789552,NaN,NaN
ACACGCGATATAACATTCGTT_TTCCTCTTCGTTCTGCTAATT___BIO_ddseq_1.FIXEDCELLS,3.597366,3.681241,4800,3957,4.434553,3.819215,27199,6595,20604,0.757528,16613,3955,0.599697,40.580825,BIO_ddseq_1.FIXEDCELLS,ACACGCGATATAACATTCGTT_TTCCTCTTCGTTCTGCTAATT,0.163511,False,Cytotoxic T cell,0.581627,7,Cytotoxic T cell,-0.056257,5.810438,-13.460642,31.882443,NaN,NaN
AACGGTGGAGAGGTTAGTGTT_TTGTAAGCGTTTGATGAGGAG___BIO_ddseq_1.FIXEDCELLS,3.850524,3.967782,9285,7088,4.727330,4.125741,53374,13358,40016,0.749728,34227,8510,0.637071,29.227156,BIO_ddseq_1.FIXEDCELLS,AACGGTGGAGAGGTTAGTGTT_TTGTAAGCGTTTGATGAGGAG,0.283820,False,CD4+ T cell,0.680369,12,Cytotoxic T cell,-1.336349,1.076014,-10.645339,3.344298,NaN,NaN
ATAGTTGTGAGATTGAATCAA___BIO_ddseq_1.FIXEDCELLS,3.683317,3.776411,5976,4823,4.381927,3.901295,24095,7967,16128,0.669350,14783,4961,0.622694,29.081667,BIO_ddseq_1.FIXEDCELLS,ATAGTTGTGAGATTGAATCAA,0.300000,False,B cell,1.000000,3,B cell,9.337917,-1.678898,15.481422,-39.594394,NaN,NaN
TACGCATTCTGAACGAGCGTG___BIO_ddseq_1.FIXEDCELLS,3.658107,3.740600,5503,4551,4.313952,3.876102,20604,7518,13086,0.635119,10367,3903,0.519154,33.464056,BIO_ddseq_1.FIXEDCELLS,TACGCATTCTGAACGAGCGTG,0.283820,False,CD4+ T cell,0.916906,18,CD4+ T cell,-1.358059,-1.996953,-24.730488,-19.812250,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GACGAGGTAAGATGGCCAAC___VIB_hydrop_22.FIXEDCELLS,2.828015,2.883093,764,673,3.219846,3.011147,1659,1026,633,0.381555,863,541,0.527290,35.755000,VIB_hydrop_22.FIXEDCELLS,GACGAGGTAAGATGGCCAAC,0.061224,False,Cytotoxic T cell,0.677033,5,Cytotoxic T cell,-5.841873,10.412942,-25.110930,-22.789226,SNG,sampleA
AGGTTGCATTATCCGAGTAT___VIB_hydrop_22.FIXEDCELLS,2.781037,2.961895,916,604,3.386321,3.064083,2434,1159,1275,0.523829,1212,590,0.509060,23.245000,VIB_hydrop_22.FIXEDCELLS,AGGTTGCATTATCCGAGTAT,0.114754,False,Cytotoxic T cell,0.517525,16,Cytotoxic T cell,-5.213475,8.696330,-22.000815,-9.940833,SNG,sampleA
TCAAGAGGCGAGGACGTTCG___VIB_hydrop_22.FIXEDCELLS,2.986772,3.149835,1412,970,3.632457,3.265996,4290,1845,2445,0.569930,2452,1044,0.565854,43.741158,VIB_hydrop_22.FIXEDCELLS,TCAAGAGGCGAGGACGTTCG,0.114754,False,Cytotoxic T cell,0.538525,8,Cytotoxic T cell,-6.212889,8.207357,-33.741268,-4.946876,SNG,sampleB
GGAGTATTCTCAAGACGTCT___VIB_hydrop_22.FIXEDCELLS,3.048442,3.171726,1485,1118,3.788239,3.291369,6141,1956,4185,0.681485,3585,1106,0.565440,41.785295,VIB_hydrop_22.FIXEDCELLS,GGAGTATTCTCAAGACGTCT,0.097345,False,Cytotoxic T cell,0.440256,4,CD4+ T cell,-4.861959,6.946391,-26.550515,6.269226,SNG,sampleB
